# Scrapper Enhancer

**Purpose of this notebook:** Create a JSON structure for the most complex pages in strengthlog.

In [10]:
import os
import json
from IPython.display import HTML

import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

In [11]:
# Available Gemini models
gemini_latest = "gemini-1.5-pro-latest" # Latest version, only for testing or prototyping
gemini_latest_stable = "gemini-1.5-pro" # Latest stable version, ready for prod
gemini_flash_latest_stable = "gemini-1.5-flash"

### Helper functions

In [21]:
def html_string(exercise_html):
    with open(f"data/html/{exercise_html}", 'r') as file:  # r to open file in READ mode
        html_as_string = file.read().encode('raw_unicode_escape').decode('unicode_escape').encode('utf-16_BE','surrogatepass').decode('utf-16_BE')
    return html_as_string

def read_json_as_string(json_file_path):
    """Reads a JSON file and returns its content as a string.

    Args:
    json_file_path: The path to the JSON file.

    Returns:
    A string containing the JSON content.
    """

    with open(json_file_path, 'r') as f:
        json_data = json.load(f)
    return json.dumps(json_data, indent=2)

# Low-bar safety filters
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

### Step 1 - Convert HTML content to valid strings

In [22]:
air_squat_text = html_string('air-squat.html')
band_assisted_bench_press_html_text = html_string('band-assisted-bench-press.html')
bench_press_html_text = html_string('bench-press.html')
cable_chest_press_html_text = html_string('cable-chest-press.html')
deadlift_html_text = html_string('deadlift.html')
dumbbell_curl_text = html_string('dumbbell-curl.html')
hip_thrust_text = html_string('hip-thrust.html')
kettlebell_swing_text = html_string('kettlebell-swing.html')
squat_html_text = html_string('squat.html') 

/var/folders/32/gbbjsz7x4wq89kgmqg3whkz800vn8t/T/ipykernel_55817/3957142109.py:3: DeprecationWarning: invalid escape sequence '\/'
  html_as_string = file.read().encode('raw_unicode_escape').decode('unicode_escape').encode('utf-16_BE','surrogatepass').decode('utf-16_BE')


### Step 2: Generate JSON from HTML text

#### V1 - HTML2JSON in one only step

In [24]:
def gen_json_from_html(exercise_html_text):
    vertexai.init(project="testing-elena", location="europe-west4")
    model = GenerativeModel(gemini_latest_stable)
    responses = model.generate_content(
        ["""
        You are a concise generator of valid and complete JSON files. You need to generate a JSON file corresponding to an HTML file, describing information on how to perform a certain exercise.
        Generate the complete and valid JSON and make sure all useful information is included.
        Based on the useful text from a reference HTML containing information about an exercise, you need to product a JSON containing the following information (1-4).
        Ignore information not needed, and also ignore images, videos, calculators, and other things that are not useful text. Ignore also all content transmitted to powerlifters, or around powerlifting.
        Only include information related to powerlifting if it's a fun fact or pro-tip (comments section) that could be interesting for the average strength exercise user.
        Be brief, give specific indications and recommendations -as oposed from redundant and repeating information-. Ignore personal viewpoints from the original HTML. 
        Place JSON line breaks (\n) when needed, e.g. to separate steps or paragraphs in one same section or item inside the JSON file.

        Sections to be inlcuded from HTML text:
        
        1. Form:
        1.1. Steps: Describes step by step how to perform an exercise.
        1.2. Form: Describes with the best detail possible all the movements involved in the exercise in order to perform it with the proper form. Separate each different aspects to explain on the form in a different key-value pair.
        
        2. Common mistakes: Contains typical mistakes that users make when performing this exercise.
        
        3. Comments: This section can contain as many sub-sections as interesting topics can be commented on this exercise. Separate each comment in a different key-value pair. Some examples could be:
        - Different ways or positions to perform the exercise, and which muscles get more involved in each.
        - Pro tips of any kind
        - Basically any information that can be valuable for the user and is not covered in any other section.
        
        4. Alternatives and variations: Other exercises that are variations of this (e.g. with different material) or other exercises train similar muscle groups, together with the main difference with this one.
        
        Then, basen on your own knowledge, complete the last section:
        5. Tempo. Tempo describes the speed of each phase of a repetition. It's usually written as a four-digit number (e.g., 2010). Here's what each digit represents:
            - 1st digit: Eccentric phase (lowering the weight)
            - 2nd digit: Pause at the bottom of the movement
            - 3rd digit: Concentric phase (lifting the weight)
            - 4th digit: Pause at the top of the movement
        In this section, you need to set the ideal tempo (could be a range) to perform this exercise and describe it (the Why), based on your own knowledge.
        
        Respond with just a valid JSON file. Do not include any additional notes or explanations.
        
        Examples:
        
        1. HTML:
        """
        , band_assisted_bench_press_html_text,
         
        """   
        1. Your response:
        "band-assisted_bench_press": {
            "form": {
                "Steps": "1. Attach the resistance band under the bench, loop it around the barbell, and back under the bench., \n2. Lie on the bench with feet flat, back slightly arched, and shoulder blades retracted., \n3. Grip the barbell at your preferred width., \n4. Unrack the barbell and position it above your chest., \n5. Lower the barbell to your chest, keeping tension in your upper body., \n6. Press the barbell back up to the starting position against the band., \n7. Repeat for reps.",
                "Form": "Core and Posture: When you’ve jumped/climbed up to the starting position, make sure to embrace your core and to have a good posture. Keep your shoulders down and back. If you don’t keep the tension in your core, the movement will be much more unstable and wobbly. \nLegs: By crossing your ankles and having a slight bend in your knees, you’ll help to make the movement even more stable. Movement: Make sure to do the entire movement with full control. Do not rush, and keep your form strict. Your elbows should go backward, not out, and your shoulders should stay down and back. You should adapt the weight to your strength level: If the bar dip is too heavy for you, you can adjust the weight by taking a resistance band and placing it around your wrists. Let the band hang between your arms, forming a U shape. Place your feet or knees on the band, which will help you on the way up. If the exercise is too easy, you can add extra weight by getting a weight belt and attaching weight plates to it.  \nHand position: There are two major points to consider regarding how you grip the bar: 1. Your thumb must be on the opposing side of your other fingers, meaning that you grip around the bar: {grip_with_thumb}. I strongly recommend that you avoid using a thumbless grip (this is having the thumb in the same side than all fingers: {grip_suicide}). Firstly, it is not allowed in powerlifting. More importantly, it is also known as the suicide grip. This is because you can easily slip and drop the bar with a high risk of injury. Don’t use it. 2. The bar should rest in a comfortable place. As you’re lifting heavier and heavier weights, this might become tricky. For most people, the most comfortable and stable position is to hold the bar as close to the base of your palm as possible, i.e. close to or directly above your wrist. This minimizes the leverage the bar works on your wrist and makes it easier to hold. Combining the two points (keeping your thumb around the bar, while still keeping it low in your hand) above might be tricky, and one solution is to slightly rotate your hands inwards so that the barbell rests in a diagonal line in your palm. Don’t keep a completely straight wrist. This puts you at risk of losing balance, tilting your wrist forward, and dropping the barbell. Instead, tilt your wrist back slightly, to better rest the barbell in your hand. \nElbow Position: Where should your elbows be during the lift: close to your body or out to the sides? It depends on your grip width, your anatomy, and also your purpose with bench pressing. Grip width generally determines elbow position, with a wide grip typically leading to your elbows pointing out, and a close grip leading to your elbows being closer to your sides. If you bench press in order to grow your chest muscles, a rather wide elbow position and a wide grip is effective. If you bench press in order to train your triceps, a close grip with your elbows close to your sides is effective. If you want to lift as much weight as possible, something in between (around 45° to your sides) is probably best. You will, however, have to experiment for yourself and see what feels the best and suits your body. \nShoulder Blades: Your shoulders are mobile joints. But when you’re bench pressing, you want to trade some of that mobility for stability. Therefore, you will want to contract the scapulae (pull your shoulder blades down and back before you unrack the bar), and keep them in this position throughout the lift. If you don't know what this means, visualize that you’re putting them in your back pockets, but on the opposing side, i.e. your right shoulder blade into your left back pocket, and vice versa. Note that you will have to “set” your shoulder blades in this manner before you even unrack the bar, because when you’ve got a heavy bar in your hands, it’s too late. Then, your shoulder blades stay in this position until you re-rack the barbell again. That means you will keep your shoulder blades retracted and pulled down even as you’re pressing the bar up, which might require some practice. Pulling your shoulder blades down and back and keeping them in this position during your entire set will protect your shoulder joints and provide you with a solid base, enabling you to lift more. E: CONTINUE WITH FOOT POSITION."
            },
            "tempo": "2110 or 3010: 2 seconds eccentric phase (lowering the weight), 0-1 second at the bottom, 1 second for lifting (concentric phase). Why? The band helps most at the bottom of the lift, so a slightly slower eccentric (2-3 seconds) helps you control the weight and maximize muscle engagement throughout the range of motion. A brief pause at the bottom reinforces good form.",
            "commonMistakes": "1. Not keeping your shoulders down. By letting your shoulders shrug up towards your ears, you’ll make the movement less efficient, increase the stress on the shoulder joint, and could cause discomfort in the neck and shoulder area rather than exercising the targeted muscles. \n2. Half reps. Since the bar dip is quite a heavy exercise, it’s easy to start to cut off depth to be able to do more reps. Don’t do this. Focus instead on maintaining a full range of motion (in line with your mobility). If the movement is too heavy, add a resistance band to reduce some of the weight. \n3. Not maintaining form. Using momentum, swinging the body to complete the movement, arching your lower back, and/or flaring your elbows out to the sides, rather than relying on controlled, targeted muscle activation, is a common mistake while doing bar dips.",
            "comments": "The bench press against bands is a variation of the bench press in which resistance bands increase the tension as you press up. This makes it more challenging to lock out the lift, enhancing strength and power. It’s typically used to improve explosive strength and stability. Make sure to use the safety racks or to have a spotter while performing this exercise.",
            "alternativesAndVariations": "1. Bench Dip (puts more emphasis on your triceps, it's usually a bit easier). \n2. Bench Press: one of the most popular exercise, could be a good substitute if having shoulder pain when performing bar dips. \n3. Close-Grip Push-Up (more emphasis on your triceps)."
        }      
        
        2. HTML:
        """
        , cable_chest_press_html_text,
        """       
       
        2. Your response:
        "cable_chest_press": {
            "form": {
                "Steps": "1. Fasten a pair of handles in the top position of a cable cross. Grip the handles, step forward, and lean slightly forward. \n2. Push the handles forward until they meet in front of your body. \n3. With control, let the handles go back to the starting position.",
                "Form":" "
            },
            "tempo": "2010. Why? Cables provide constant tension, so a 2-second eccentric and smooth transition to the concentric maintain that tension for optimal muscle growth.",
            "commonMistakes": " ",
            "comments": "Cable chest press is a pressing exercise with a different resistance curve than for example dumbbell chest press, as the direction of resistance comes more from the sides. That might make it easier to keep tension on the chest muscles throughout the whole movement, and thus possibly benefit muscle growth.",
            "alternativesAndVariations": "Dumbbel chest press, Bench press and all its variations."
        }
        
        
        3. HTML:
        """
        , bench_press_html_text,
        """
        
        3. Your response:
        "bench_press": {
            "form": {
                "Steps": "1. Lie on the bench, pull your shoulder blades together and down, and slightly arch your back. \n2. Grip the bar slightly wider than shoulder-width apart. \n3. Inhale, hold your breath, and unrack the bar. \n4. Lower the bar with control, until it touches your chest somewhere close to your sternum. \n5. Push the bar up to the starting position while exhaling. \n6. Take another breath while in the top position, and repeat for reps.",
                "Form": "Position on the Bench. In most bench press federations, the back of your head, your upper back, and your glutes must remain in contact with the bench at all times, and your feet must touch the floor. You should place yourself close enough to the bar so that you can easily unrack it. If you are too close to the foot end of the bench, unracking the bar will be excessively heavy. If you are too close to the head end, you will hit the rack with the bar when you’re lifting. A good rule of thumb is to lie down so that your eyes or nose are directly below the racked bar. This will be close enough for you to unrack it easily, but not close enough to hit the rack while you’re benching. If you’re tall, you can probably slide even closer to the head end without hitting the rack, and if you’re short, you might have to be a little bit closer to the foot end. \nHand position: There are two major points to consider regarding how you grip the bar: 1. Your thumb must be on the opposing side of your other fingers, meaning that you grip around the bar. 2. The bar should rest in a comfortable place. As you’re lifting heavier and heavier weights, this might become tricky. For most people, the most comfortable and stable position is to hold the bar as close to the base of your palm as possible, i.e. close to or directly above your wrist. This minimizes the leverage the bar works on your wrist and makes it easier to hold. Combining the two points (keeping your thumb around the bar, while still keeping it low in your hand) above might be tricky, and one solution is to slightly rotate your hands inwards so that the barbell rests in a diagonal line in your palm. Don’t keep a completely straight wrist. This puts you at risk of losing balance, tilting your wrist forward, and dropping the barbell. Instead, tilt your wrist back slightly, to better rest the barbell in your hand. \nShoulder Blades: Your shoulders are mobile joints. But when you’re bench pressing, you want to trade some of that mobility for stability. Therefore, you will want to contract the scapulae (pull your shoulder blades down and back before you unrack the bar), and keep them in this position throughout the lift. If you don't know what this means, visualize that you’re putting them in your back pockets, but on the opposing side, i.e. your right shoulder blade into your left back pocket, and vice versa. Note that you will have to “set” your shoulder blades in this manner before you even unrack the bar, because when you’ve got a heavy bar in your hands, it’s too late. Then, your shoulder blades stay in this position until you re-rack the barbell again. That means you will keep your shoulder blades retracted and pulled down even as you’re pressing the bar up, which might require some practice. Pulling your shoulder blades down and back and keeping them in this position during your entire set will protect your shoulder joints and provide you with a solid base, enabling you to lift more. \nFoot position: The goal of your foot position is to keep you stable and support you throughout the lift. You will want to place your feet firmly on the floor and keep them still throughout the lift. Don’t fidget with your feet, as that will rob you of any stability. Don’t move your feet around after you’ve unracked the bar. Plant them, unrack, and then keep your feet still until you’ve racked the bar again. An exception is if you go up on your toes when you’re unracking, but in that case, plant your feet on the floor again as soon as you’re in position. \nLeg Drive (pro): Leg drive simply means that you apply a slight push with your legs. This will contract and tense your leg muscles, which in turn makes your whole body a little harder and more stable. And as the bench press is a lift that thrives on stability, you will likely be able to lift a little more weight because of this. To use leg drive in the bench press, push with your legs like if you tried to slide yourself over the head end of the bench. Meaning, you should apply force horizontally with your feet, away from the foot end of the bench. Note that you should only apply just enough force so that your feet don’t move. Your feet should still be completely still throughout the entire lift. You’re only pushing just enough to stay within the limits of friction, in order to tense your legs and stabilize your body."
            },
            "tempo": "2010 or 3010. Why? A controlled eccentric is key for proper form and muscle activation. You can experiment with a 2- or 3-second eccentric based on your strength and experience.",
            "commonMistakes":"1. Use a thumbless grip (this is having the thumb in the same side than all fingers). Firstly, it is not allowed in powerlifting. More importantly, it is also known as the suicide grip. This is because you can easily slip and drop the bar with a high risk of injury. Don’t use it. \n2. Round shoulders instead of contracting the scapulae (right position is with shoulder blades down). \n3. Push your feet down into the floor when using leg drive. This will only lead to you lifting your butt off the bench, which is, once again, not allowed in competition and not a recommended technique. ",
            "comments": {
                "Popularity":"The barbell bench press is one of the most popular exercises in the world and one of the best chest exercises you can do",
                "How Wide Should You Grip the Bar?": "Your correct grip width will depend on your anatomy, and also your goal with the exercise. Typically, people are stronger (can lift more weight) with a wider grip than with a narrower grip. The wider you grip the bar, the greater is the contribution of your chest and front delts. Conversely, the narrower you grip the bar, the greater is the contribution of your triceps. In a wide-grip bench press, your chest and front delts contribute 78% of the force necessary to lift the bar, while your triceps contribute 22%. In a close-grip bench press, the same numbers are 63% vs. 37%. The maximum grip width allowed in powerlifting is 81 cm between your hands. A powerlifting bar has rings marking this distance, and so your maximum grip width is when your index fingers cover these rings.",
                "Elbow Position: Where should your elbows be during the lift, close to your body or out to the sides?": "It depends on your grip width, your anatomy, and also your purpose with bench pressing. Grip width generally determines elbow position, with a wide grip typically leading to your elbows pointing out, and a close grip leading to your elbows being closer to your sides. If you bench press in order to grow your chest muscles, a rather wide elbow position and a wide grip is effective. If you bench press in order to train your triceps, a close grip with your elbows close to your sides is effective. If you want to lift as much weight as possible, something in between (around 45° to your sides) is probably best. You will, however, have to experiment for yourself and see what feels the best and suits your body",
                "Pro tip on Foot position": "Place your feet somewhere where you feel comfortable, stable, where you are able to produce some manner of leg drive. If you don’t know where to start, try placing your feet about hip-width apart, with your legs bent at a 90-degree angle or slightly more.",
                "Should You Pause With the Bar on Your Chest, or “Touch and Go”?": "If you don’t train for powerlifting, pausing in the bottom position is optional. It probably doesn’t have any additional benefits for building muscle compared to a controlled reversal. It might, however, help you improve your technique because you slow things down and get more practice in one of the hardest parts of the lift. Similarly, pausing can improve your strength off your chest. Bench pressing without a pause on your chest is called touch and go. You lower the bar down and touch your chest, without neither pausing nor bouncing, and then press the bar up again. Avoid bouncing the bar on your chest. It puts you at risk of injuring your ribs or your sternum, which can put a stop to your training for a long time.",
                "Pro tip:": "Push-ups work the same muscles as the bench press (= your chest, front deltoid, and triceps) in a similar movement, and can therefore be used as an bench press accessory exercise."
            },
            "alternativesAndVariations": "1. Close-Grip Bench Press, which puts more emphasis on your triceps. \n2. Feet-Up Bench Press, which greatly inhibits the amount of arch you can create with your lower back, and thus forces you into a more flat bench press technique. This leads to a longer range of motion, which, in turn, might benefit both your strength gains and muscle growth. \n3. Close-Grip Feet-Up Bench Press, combining Close-Grip Bench Press and Feet-Up Bench Press. Greater triceps emphasis, more training for your lockout, and stability training. \n4. Incline Bench Press. Inclining the bench emphasizes your upper chest muscles and anterior deltoids. Typically, a slight decline of about 30° is enough to get this effect. A popular variation of the incline bench press is to trade the barbell for a pair of dumbbells and perform incline dumbbell presses instead. Many people feel that the freedom of motion of the dumbbells enable them to lift in a more comfortable movement path. \n5. Smith Machine Bench Press. Doing bench press in a Smith Machine has pros and cons. On the pro side is the extra stability, which might enable you to work your muscles a little harder than usual because you don’t have to worry about balance. On the negative side, we have the fact that a normal, free barbell bench press doesn’t move in a vertical bar path (see the section on bar path for more info). The smith machine, however, forces you to work in a straight line, which an experienced bench presser might perceive as uncomfortable."
        }
        
        
        4. HTML:
        """
        , deadlift_html_text,
        """
        
        4. Your response:
        "deadlift": {
            "form": {
                "Steps": "1. Step up close to the bar so that it is about over the middle of your foot. \n2. Bend over and grip the bar with hands about shoulder-width apart. \n3. Inhale, brace your core slightly, and lift the bar by extending your hips and knees. \n4. Pull the bar close to your body, with a straight back, until you are standing straight up, with hips and knees locked out. \n5. Lower the bar back to the ground with control. \n6. Take another breath, and repeat for reps.",
                "Form": {
                    "1. Foot Position": "A rule of thumb is to point your toes in the same direction that your knee travels. This is not set in stone, however, and you still have some leeway to divert from this. You can choose to place your feet all the way together until your heels are touching. This will probably also require you to point your toes out a bit. More common and the best fit for most people, however, is to simply place the feet directly under your hip joints, with your toes pointing straight forward or only slightly out. What you’ll want to avoid is a stance so wide that your knees push your arms out. You’ll want your arms to be perpendicular to the floor as seen from the front, as this will result in the shortest and strongest range of motion. With a wide grip, you’d have to lean over more to reach the bar, resulting in a weaker starting position. It is a good idea to experiment a bit with different foot positions when you’re learning the deadlift, as your individual limb lengths will dictate what position suits you better.",
                    "2. Arms": "The best visualization of your arms’ roles in the deadlift is to act like ropes, attaching the bar to your shoulders. Preferrably, these should also be both long and straight ropes, as that will allow you to maximize your starting height in the lift. The first few inches of a deadlift is the biomechanically weakest part of the lift, and the more you can extend your knees and hips from the start, the easier it will be to pass this point. If you deadlift with your arms bent or excessively wide, you will have to lean over more, and thus end up in a weaker starting position.",
                    "3. Grip": "How to grip the bar? There are 4 common ways: \n1. Double overhand grip. This is the standard, intuitive way to grab a barbell. You simply hold on to it with both palms facing your body (pronated), and the thumbs opposing the other fingers. Pros: Comfortable and intuitive. The way you usually grab stuff. Symmetrical for your arms and shoulders. Cons: Weak! The barbell will want to roll out of your grip. This grip technique is usually fine for beginners learning the deadlift form, but quickly becomes a greatly limiting factor for how much you can lift. \n2. Hook grip. Common in weightlifting and  an evolution of the double overhand grip. Despite them being very similar, the hook grip increases the weights you can lift by 20–30% or more. You still grab the bar with a double overhand grip, with the difference that you stick your thumb under your index finger (and maybe also your middle finger). The thumb will act like a wedge or a chock and hinder the bar from rolling out of your grip. Pros: Much stronger than a regular overhand grip. Symmetrical for your arms and shoulders. Cons: Painful! It gets better with time, but will still be more uncomfortable than the normal way of gripping a bar. \n3. Mixed grip. Most common grip technique in powerlifting. Here, you counter the barbell’s rotation by supinating one hand – that is, turning one palm facing away from you. Like the hook grip, this greatly increases the weight you can lift. Pros: Much stronger than a regular overhand grip. Not very painful or wearing on your skin (compared to hook grip). Con: Asymmetrical for your arms and shoulders, which might transfer to an asymmetrical spine and hip position. It is unclear, however, if this actually is a problem. \n4. Lifting straps (not allowed in most powerlifting competitions). Pros: Using lifting straps will remove any grip related limitations for your pulls, will spare your skin, and also enable you to pull with a symmetrical overhand grip. Cons: 1. You won’t reap the benefit of developing your grip strength during training. 2. One more thing to carry around in your gym bag.",
                    "4. Hip position": "How high should your hips be in the starting position of a deadlift? Should you begin in more of a “squat” position, or something more along the lines of a stiff-legged deadlift? The answer can be deduced from three instructions: 1. Keep your arms vertical as seen from the side (or very close to vertical) during the entire lift. This is easy: once the bar gets heavy enough, it’ll be positioned right below your shoulder joint, or at least very close to it. 2. Keep the systems (your body + the bar = the system) center of mass roughly above your mid foot. How do you find this position? It’s the position in which you don’t lose your balance forward or backward during the pull – e.g. where you’ll automatically end up positioning the bar to keep your balance, without thinking about it. You can explore and reinforce this position by performing a pause deadlift, with the pause very low towards the floor. 3. Keep your shins as close to the bar as possible while still allowing for number 1 and 2 above, as that will enable you to more effectively use both your knee and hip extensors. For many people, this means having your shins touching the bar. These three points will have narrowed down your possible hip starting height quite a lot, but there is still some leeway regarding the exact height. The final adjustments of your hip position will depend on your personal strengths and weaknesses, especially in your back, hip and knee extensors. If you have a strong back and strong hip extensors but weak knee extensors, you’ll be the strongest (and naturally drift towards) when you are a little bit more bent over and thus shifting more of the load towards your hip. If you have strong knee extensors but a weaker back or weak hip extensors, you’ll be the strongest (and naturally drift towards) when you have a more vertical torso and your shins/knees closer to the bar (or even past the bar, for taller lifters). Note that your ideal starting position probably is one that you already know and use. When we perform movements, we usually experience a bit of micro-variation. The movement simply doesn’t look exactly the same each rep, and if we by chance discover a slightly easier and more efficient way of performing a rep, we usually take notice – either consciously or subconsciously. That is why, as long as you’re not doing something fundamentally wrong – like keeping the bar way ahead of your shins – your current starting position is probably also your most effective one. At least right now. As your body and musculature changes with continued training, so will your strengths and weaknesses change. That means that the most effective deadlift position in the beginning of your lifting career, might not be the same as your most effective lifting position when you are on your peak. Do experiment with a slightly higher or lower hip position in the start – although my bet is that you’ll find that you’re either weaker with the new position, or will end up in your old position anyway when the load gets heavy enough.",
                    "5. Back position & Bracing":"In general, you should deadlift with a neutral spine. Generally avoid lifting with a flexed back in your training, as it could increase your risk of injury – if you cannot maintain a neutral spine, lower the weight or perform less reps per set. To maintain a neutral spine, you can implement bracing, which refers to the act of keeping your core rigid during a lift. Start by taking a deep breath just before lifting, about 80% of maximum. Then, it’s time to tense up your core muscles. Effective deadlifting requires efficient force transfer. The force must be transferred from the bar, via your grip, arms and shoulders, through your torso down to your hips, and finally down via your legs and feet to the floor. A key to effective force transfer is stiffness, and this ties in with the previous section on bracing. Arms and legs won't heavily help here.  The core however, with it’s pelvis connected to the spine  is in great need of stabilization – especially in the conventional deadlift since it places greater demands on your low back. A braced and stiff core will be essential for effective force transfer. Before you push the pedal to the metal and revv up those strong prime movers in your back, hip and knee extensors, you’d better take a moment to make sure your core is good to go, and this is done by building tension. It is simply the act of consciously tensing your body to prepare it for what is about to come.",
                    "6. Lift-Off":"The lift-off is really about nailing down two things: 1. Knowing your strongest pulling technique, and 2. Using and maintaining that technique. Your own strongest pulling technique will entail things like: how you distribute work between hip and knee extensors, what sequence and speed you will extend different joints in, and whether or not you are going to pull with a rounded or straight back. The deadlift is more segmented into three distinct phases: 1. Mostly knee extension, 2. Mostly hip extension, 3. Simultaneous knee and hip extension until lock-out. Some generally  helpful cues for lift-off: Push the floor away. Instead of pulling the barbell up, you focus on pushing the floor away, or down. Chest up, this is one of the best ways to help a lifter maintain a more straight back.",
                    "7. Lock-Down":"Assuming you are strong enough to complete the lift, the lock-out part of a deadlift is  pretty simple. All you have to do is: extend your knees, hips and back, and keep your shoulder back for the lift to be considered completed. Simply drive your hips forward, kind of like a hip thrust. If you’re having troubles extending your spine in the top or having “soft knees” (not fully extended), a helpful visualization might be to imagine a chain attached to the top of your head, pulling you upwards.",
                    "8. Setting the Bar Down":"How to set the bar down safely. acktrack your steps from the pull, even though we compress it slightly into two distinct phases: 1. Hip hinge. Initiate the eccentric by pushing your butt back and leaning your torso forward. The knees bend, but not as fast as your hip. Lower the bar along your legs like this until you’ve cleared the knees. 2. Knee bend. When the bar has cleared your knees, you are probably leaning as far forward as you’re going to get. At this point you take out the rest of the movement with your knees, and simply bend them until the bar touches the ground."
                }
            },
            "tempo": "11X1: 1021, 1031, oe 1041: 1 second to lift the weight explosively, 1 second to pause at the top, and control the bar on the way down for X seconds (2-4 seconds depending on the weight and individual preference). Why? The explosive lifting phase (1 second) recruits the maximum number of muscle fibers and develops power. Controlling the weight on the way down (eccentric phase) helps maintain tension in the muscles, promotes hypertrophy, and reinforces good form. The brief pause at the top prevents loss of tension and ensures continuous muscle engagement. The variable tempo (X) on the eccentric phase allows for individual adjustment based on experience level and training goals. Beginners may benefit from a slower descent (4 seconds) to focus on technique and control, while experienced lifters may use a faster tempo (2 seconds) to increase training volume and intensity.",
            "commonMistakes": {
                "Feet & Legs stance - Have  the stance is so wide that the knees push the elbows out":"This forces you to lean forward, thus bringing you closer to the floor, and into a biomechanically weaker position. Avoid this if possible. It may be, however, that your thighs and abdomen are too big for a narrow stance, and thus forcing you to adopt a stance similar to this.",
                "Arms - Bend the arms or grip the barbell excesively wide":"This will make you have to lean over more, and thus end up in a weaker starting position, making it harder to start the movement and increasing the injury risk.",
                "Hip position":"[E - Leave only if image present] Here (image) I am trying to show different hip positions. My anatomy doesn’t change between the pics, so only one of these three are really a viable option for me. The left pic with a high hip position is just fake posing: note the angle of the arm. As soon as I would try to lift a heavy weight that way, my arm would swing out to an angle perpendicular to the direction of gravity, and the moment arm from the weight to my hips would be very long and ineffective for heavy pulls. In the right pic with a low hip position on the other hand, the knees would be in the way of the bar. My personal starting position is the middle one. Just right for me."
            },
            "comments": {
                "Conventional or Sumo Deadlift?": "\n",
                "Discover your ideal Lift-Off position":"You will have to experiment to find out for yourself: Are you stronger when you let your butt shoot up in the start, or when your patiently push with your legs until the bar finally gives? Are you stronger when you deadlift with a straight back during the whole lift, or when you initally round your back but re-extend it during the lock-out? Your individual optimal deadlift technique will vary, but if we yet again shall revert to generalizations, we believe that most lifters will eventually have their greatest performances when they patiently let their quadriceps contribute to the lift, resulting in a pronounced knee extension in the start, as well as pulling with a more or less straight back through the first phase of the lift, thereby ridding themselves the task of re-extending their spine. So, how do you go about accomplishing this? Both depend on grooving the right neuromuscular motor patterns, which in plain English means train a lot. With the desired technique. With progressively heavier weights, as you get increasingly good at maintaining your technique with heavier weights. To train this, you will be needing some kind of feedback to discern whether you’re doing it right or wrong: A coach/friend/GymAlly who knows what to look for might give you feedback after a set or even after every rep.",
                "Strength advantage to controlled Eccentric":"Is there a strength advantage to setting the bar down slowly in your training (controlled Eccentric)? The tricky part of a deadlift is the way up – the concentric phase. That is what you compete with in powerlifting, and the way down (the eccentric phase) is mostly a formality, where the only rule is to lower the bar with control and maintaining your grip on it until it rests on the floor. However, for general training, it's good to know that several studies has compared the concentric strength gains from training protocols where you either perform concentric reps only or concentric+eccentric (that is, normal training) reps, and found that concentric+eccentric training yields greater concentric strength gains. This means that even if you perform twice as many reps where you set the bar down quickly (without really braking it’s fall), you’re still not getting as good gains as if you were to really slow down the bar, and lower it under, say, at least two seconds. Thus, a little bit more work and discomfort invested in your deadlift training in the shape of more prolonged eccentrics might pay good dividends.",
                "How to Become Stronger in the Deadlift?":"The weight you can lift in the deadlift is determined by the amount of force you can produce in your muscles (primarily the extensors of the hip, knee, and back) and transfer into the bar. To deadlift more weight, you must generate more force into the bar. There are three different domains in which you can improve in order to accomplish this: \n1. Muscle & tendon. A bigger muscle produces more force, and a stiffer tendon transfers that force into your bones more efficiently. Skeletal muscle mass is strongly correlated with deadlift performance. To build bigger muscles, you need to train them in any reasonable way. You can train them deadlifting, but also doing other exercises. E.g. If you build bigger and stronger glutes using hip thrusts, you will still have a bigger and stronger hip extensor when you go back to the deadlift. Tendons increase in stiffness from all heavy training. Improvements in neuromuscular efficiency and especially technique require more specificity, and you will actually have to practice the lift in which you want to improve. Preferrably at pretty heavy loads.",
                "Training Volume for the Deadlift":"The optimal training volume varies widely between individuals and is dependent on several factors, such as training status, age, nutrition (especially calories and protein ingestion), and recovery. The relationship between strength gains and training volume follows an inverted U-shape, where both too little and too much training leads to inferior gains. The key is to find the Optimal training volume. You’ll neither want to train too little nor too much for optimal gains. But if you have to choose, erring on the side of too little will at least spare you time and decrease your injury risk. Some general volume recommendations in terms of working sets of deadlifts. That is sets that are challenging but are generally not performed to failure but leaving 1–2 reps in the tank. \n1. Low volume, 3–5 sets per week: This is a suitable volume for the beginner lifter that is simultaneously getting used to squatting maybe 1–2 times a week and thus is acclimatizing to a lot of new stress. It might also be suitable for the intermediate lifter, either tapering or taking a minimalistic approach to deadlifting while perhaps focusing on hypertrophy work for the deadlifting musculature. \n2. Medium volume, 6–13 sets per week: The goldilocks zone, and probably a nice target volume for most people in the intermediate stage. Recovery demands will be quite low if this is the only training you do for the lower body, but if you add in squats and deadlift accessory training, the total volume will place quite high demands on your recovery. This might be a suitable volume for an advanced lifter who is tapering. \n3. High volume, 14–20 sets per week: For the intermediate lifter who is stuck on a plateau and wants his or her deadlift to ascend to the next level, or for the advanced lifter. Recovery demands are high, especially if this is coupled with squat training and accessory work. Most will have to gradually increase their volume to this level over a long time, and some lifters might find it too much for them, and that medium volume works better.",
                "Training for Muscles Used in the Deadlift":"For optimal deadlift gains, you will probably want to train the prime mover muscles more than you can do if you’d only use deadlifts, by doing assistance exercises. By varying your exercises, you can probably lower the risk of over-use injuries. Assistance exercises are usually for hypertrophy, in this case, exercises that strengthen and grow the muscles used in the lift. Major muscles worked in the deadlift: Knee extensors (Quadriceps), Hip extensors (Glutes, Hamstrings, Adductors), and Back extensors (Lower back). The options for hypertrophy exercises are many and more a matter of personal preference. However, here is a sample list of exercises that you might use to supplement your training for the muscles involved in the deadlift. If you are hitting squats and deadlifts hard, you might only want to pick 1–2 of the exercises below and do 2–3 sets x 1–2 workouts per week. If you deadlift with a low volume (relative to your training level), you can increase the volume of assistance exercises. Sample exercises: Squat (if you’re not already doing them) (knee, hip and back extensors), Bulgarian split squat (knee and hip extensors), Hip thrust (hip extensors), Kettlebell swing (hip and back extensors), Hyperextension (hip and back extensors), Good morning (hip and back extensors), Romanian deadlift (hip and back extensors), Barbell row (upper back and isometric training for back and hip extensors), Farmers walk (core, grip, trapezius, and stability training for hips)",
                "Training Frequency for the Deadlift":"Training frequency might be an important mediator for training volume. Do you want to train a lot? Well, sooner or later you’ll have to distribute the training on more days, before your current workouts swell beyond the scope of what is productive to do in one session. A rough limit on the number of work sets of deadlifts in a single session might be somewhere in the neighbourhood of 8-10 sets for an experienced lifter. About 1–2 deadlift sessions per week is not a bad starting point for someone who also trains squats or similar lower-body movements. Most lifters can probably do one hard deadlift session per week, while two sessions per week might require most lifters to dial down the intensity a bit. A classic recommendations for adapting to a larger workload that you might use for the deadlift is this: \n1. First increase the frequency. Go from one deadlift session to two, more or less by splitting your current training volume in two. \n 2. Then increase the volume. Slowly ramp up the volume by adding easy sets, so that you are well on the safe side of doing too much, too soon. \n 3. Finally increase the intensity (load). When you are accustomed to two deadlift sessions per week, you might start to increase the intensity of load you’re using.",
                "Load & Reps for the Deadlift":"Your strength can improve from training with both high and low loads, but when you compare them head to head, lifting heavy weights is slightly more effective if you want to get better at lifting heavy weights. There are several rep and loading strategies that work, as long as you heed the principle of specificity (i.e. lifting heavy every once in a while) and progressive overload (e.g. adding weight or reps). One basic template you could try is to Slug away at 75-80% of 1RM. That range is heavy enough to be specific training for max attempts, but not so heavy that it burns you out. It’s typically possible to accumulate a large, effective volume at this intensity. Something like 6-14 sets x 2–3 reps weekly might be appropriate for an intermediate level practitioner, divided in 1-2 sessions, and with an expected increase of 2.5kg per workout."
            },
            "alternativesAndVariations": "Sumo deadlift. "
        }
        
        
        5. HTML:
        """
        , exercise_html_text,
         
        """        
        5. Your response:
        """],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    for response in responses:
        print(response.text, end="")

In [9]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
    "response_mime_type":"application/json", # Force the model to return a valid JSON
}

In [ ]:
%%time
gen_json_from_html(hip_thrust_text)

**Pitfalls found (to be improved in next versions):**

**1. Unnecessary verbose sentences**, E.g.:
- "_First things first, ..._"; 
- "_So why am I going on about this? Because you must know that..._"
- "_There are many misconceptions regarding the core and the squat. Let’s clear them up in this section, with the help of scientific studies and biomechanical reasoning. ..._"
- "_So, this might be where someone would say: “Well if these study participants tensed their abs, they could squat even more!”. But where is the logic in that?_"
- "_The right descent speed for you might not be the same as for me, but in general, ..._"
- "_"Remember that..._"
- "_Contrary to what some might believe, ..._"
- "_Without getting into periodization practices, ..._"

**2. Too long paragraphs repeating the same idea**, e.g. 
- In Squat > Form > Balance in the Squat > "_(...). If you weigh 200 pounds and the bar weighs 200 pounds, your combined centre of gravity will be halfway from your belly button (your centre of gravity) to the bar. (...)_". E.g. 2. In the same section "_You will not need to have a trainer check this from the side, nor video yourself – if you’re not keeping the combined centre of gravity over the middle of your foot, you will fall over._", could be changed by "_If you'd not keep the combined centre of gravity over the middle of your foot, you would fall over._"
- In Squat > Form > "The Sticking Point and Lock-Out": "_... From a personal standpoint, this is in line with my experience as a powerlifter, having both competed at and watched numerous competitions: ..._"

**3. Sections placed in the wrong section**

**3.1. Parts in the Form section, that should be in the Comments instead**, as they are things not needed to "perform the exercise in the right way". E.g.
- Squat > Form: _"Shoes: Flat or Raised Heel?": "..._ -  Includes pros and cons of both and which one is better for each person, but it's not that only one is acceptable)".
- Squat > Form: _"The Core Muscles and the Squat": "..."_ - Explains how we shouldn't over-analyze the art of bracing during squat, but this is not something we could correct from seeing the video.
- Squat > Form > "_The Descent: (... should be in tempo...). “As fast as you can while still maintaining control.”. In general, if your descent takes ages you are wasting energy and losing your rebound effect at the bottom. If you find that you descend very slowly and want to increase your speed, one way to address that is to train a little more in higher rep-ranges, like several sets around 10 reps or so. (...) If you however feel that you might gain a little on the ascent if you could speed up your descent just a little bit, then it might be worth giving it some attention._" - Should be in comments
- Squat > Form > "_The Sticking Point and Lock-Out"_  - Explains why the sticking point is in a certain height.

**3.2. Parts in the Form section, that should be in the tempo one**:
- Squat > Form > "The Descent": "_Two things remain to discuss however, of which the first would be the tempo. How fast should you descend in the squat? The answer probably is: As fast as you can while still maintaining control. Why? Two primary reasons: Energy conserving. A slow eccentric costs more energy than a quick one. Rebound effect. (...). If you however feel that you might gain a little on the ascent if you could speed up your descent just a little bit, then it might be worth giving it some attention._"

**3.3. Some sections should be both in Form and in Common mistakes, to explain how to do it right, and how to do it wrong with common mistakes.** E.g.:
- Squat > Form > "Un-Racking and Re-Racking the Bar" - Explains why rack shouldn't be too high, which is a common mistake that is risky, and also how to place it right. Should be in both sections.

**4. Still including powerlifting terms, even if in the prompt it says not to.** E.g.:
- Squat > Form > "The Descent" - "You will now need to squat down to the depth that you need to reach. For the powerlifter, this will be just below parallel."
- Squat > Comments > "How Deep Should You Squat?" - "_If you are a powerlifter: Squat to regulation depth. In IPF, this is until the part of your thigh closest to your abdomen, is below the top of your knee. Generally this is just below parallel. If you are an athlete using squats to increase your sports performance: Do a part of your squats as deep as you safely can, and part of them to your sports “working depth”._".

**5. Referenes only to links, without which the sentence loses meaning**, e.g.
- Squat > alternativesAndVariations > Technique: Exercises that strengthen the movement, or improve the performance or technique of the lift.> "Also, see [Front Squat vs Back Squat](https://www.strengthlog.com/front-squat-vs-back-squat/)."

**6. NOTE THIS SHOULD BE IN THIS SAME MODEL CALL: Alternatives and variations don't appear, e.g.:**
- Squat > AlternativesAndVariations" > _"Front Squat": "Shifts the workload more to your quad_ - Only variation that appears, but there's two whole section on this: Auxiliary Exercises for the Squat, Squat Variations to Strengthen or Improve Technique.

**Conclusion:** We may want to add a second call to an LLM, to not distract the first one for correctly generating the JSON, in order to fix the verbosity mistakes and review of info being placed in the right section (+ validate JSON structure).

**Note:** "Squat" in strengthlog refers to a squat performed using a barbell. "Normal" (just with body weight) squat is called "Air Squat" here. We might want to change the names, calling "squat" to the body weight one and adding the needed extra info from the barbell one (as the explanation for this one is [pretty poor](https://www.strengthlog.com/air-squat/)), and changing the other one to "barbell_squat", or sth like that.

## V2 - HTML2JSON in two steps

- Step 1: Generate JSON including all info into its corresponding session. Don't ask the model for things like avoiding powerlifting terms, verbosity, or long paragraphs.
- Step 2: Reduce verbosity, review sections placed, and ommit content for powerlifters. 

**Note: Using Few-shot for both calls**, as I tried with just prompting and it didn't work.

In [4]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
    "response_mime_type":"application/json", # Force the model to return a JSON. Note: Not always a valid one!
}

In [5]:
def gen_json_from_html(ex_html_text):
    vertexai.init(project="testing-elena", location="europe-west4")
    model = GenerativeModel(gemini_latest_stable)
    responses = model.generate_content(
        ["""
        You are a generator of valid and complete JSON files. 
        You need to generate a complete and valid JSON file corresponding to an HTML file, including all useful information on how to perform a certain exercise.
        Based on the useful text from a reference HTML containing information about an exercise, you need to product a JSON containing the following information.
        Ignore information not needed, and also images, videos, calculators, and other things that are not useful text.
        IMPORTANT: IGNORE ALL URLS, DO NOT USE URLS
        

        Sections to be inlcuded from HTML text:
        
        1. form. Two sub-sections:
        1.1. Steps: Describes step by step how to perform an exercise.
        1.2. Form: Describes with the best detail possible all the movements involved in the exercise in order to perform it with the proper form. Usually has separate sub-sections on its own, one for each different aspect to explain on the form in a different key-value pair.
        
        2. commonMistakes: Contains typical mistakes that users make when performing this exercise.
        
        3. comments: This section can contain as many sub-sections as interesting topics can be commented on this exercise. Separate each comment in a different key-value pair. Some examples could be:
        - Different ways or positions to perform the exercise, and which muscles get more involved in each.
        - Pro tips of any kind
        - Basically any information that can be valuable for the user and is not covered in any other section.
        
        4. alternativesAndVariations: Other exercises that are variations of this (e.g. with different material) or other exercises train similar muscle groups, together with the main difference with this one.
        
        Then, basen on your own knowledge, complete the last section:
        5. tempo. Tempo describes the speed of each phase of a repetition. It's usually written as a four-digit number (e.g., 2010). Here's what each digit represents:
            - 1st digit: Eccentric phase (lowering the weight)
            - 2nd digit: Pause at the bottom of the movement
            - 3rd digit: Concentric phase (lifting the weight)
            - 4th digit: Pause at the top of the movement
        In this section, you need to set the ideal tempo (could be a range) to perform this exercise and describe it (the Why), based on your own knowledge.
        
        Respond with just a valid JSON file. Do not include any additional notes or explanations.
        
        Examples:
        
        1. HTML:
        """
        , band_assisted_bench_press_html_text,
         
        """   
        1. Your response:
        "band-assisted_bench_press": {
            "form": {
                "Steps": "1. Attach the resistance band under the bench, loop it around the barbell, and back under the bench., \n2. Lie on the bench with feet flat, back slightly arched, and shoulder blades retracted., \n3. Grip the barbell at your preferred width., \n4. Unrack the barbell and position it above your chest., \n5. Lower the barbell to your chest, keeping tension in your upper body., \n6. Press the barbell back up to the starting position against the band., \n7. Repeat for reps.",
                "Form": "Core and Posture: When you’ve jumped/climbed up to the starting position, make sure to embrace your core and to have a good posture. Keep your shoulders down and back. If you don’t keep the tension in your core, the movement will be much more unstable and wobbly. \nLegs: By crossing your ankles and having a slight bend in your knees, you’ll help to make the movement even more stable. Movement: Make sure to do the entire movement with full control. Do not rush, and keep your form strict. Your elbows should go backward, not out, and your shoulders should stay down and back. You should adapt the weight to your strength level: If the bar dip is too heavy for you, you can adjust the weight by taking a resistance band and placing it around your wrists. Let the band hang between your arms, forming a U shape. Place your feet or knees on the band, which will help you on the way up. If the exercise is too easy, you can add extra weight by getting a weight belt and attaching weight plates to it.  \nHand position: There are two major points to consider regarding how you grip the bar: 1. Your thumb must be on the opposing side of your other fingers, meaning that you grip around the bar: {grip_with_thumb}. I strongly recommend that you avoid using a thumbless grip (this is having the thumb in the same side than all fingers: {grip_suicide}). Firstly, it is not allowed in powerlifting. More importantly, it is also known as the suicide grip. This is because you can easily slip and drop the bar with a high risk of injury. Don’t use it. 2. The bar should rest in a comfortable place. As you’re lifting heavier and heavier weights, this might become tricky. For most people, the most comfortable and stable position is to hold the bar as close to the base of your palm as possible, i.e. close to or directly above your wrist. This minimizes the leverage the bar works on your wrist and makes it easier to hold. Combining the two points (keeping your thumb around the bar, while still keeping it low in your hand) above might be tricky, and one solution is to slightly rotate your hands inwards so that the barbell rests in a diagonal line in your palm. Don’t keep a completely straight wrist. This puts you at risk of losing balance, tilting your wrist forward, and dropping the barbell. Instead, tilt your wrist back slightly, to better rest the barbell in your hand. \nElbow Position: Where should your elbows be during the lift: close to your body or out to the sides? It depends on your grip width, your anatomy, and also your purpose with bench pressing. Grip width generally determines elbow position, with a wide grip typically leading to your elbows pointing out, and a close grip leading to your elbows being closer to your sides. If you bench press in order to grow your chest muscles, a rather wide elbow position and a wide grip is effective. If you bench press in order to train your triceps, a close grip with your elbows close to your sides is effective. If you want to lift as much weight as possible, something in between (around 45° to your sides) is probably best. You will, however, have to experiment for yourself and see what feels the best and suits your body. \nShoulder Blades: Your shoulders are mobile joints. But when you’re bench pressing, you want to trade some of that mobility for stability. Therefore, you will want to contract the scapulae (pull your shoulder blades down and back before you unrack the bar), and keep them in this position throughout the lift. If you don't know what this means, visualize that you’re putting them in your back pockets, but on the opposing side, i.e. your right shoulder blade into your left back pocket, and vice versa. Note that you will have to “set” your shoulder blades in this manner before you even unrack the bar, because when you’ve got a heavy bar in your hands, it’s too late. Then, your shoulder blades stay in this position until you re-rack the barbell again. That means you will keep your shoulder blades retracted and pulled down even as you’re pressing the bar up, which might require some practice. Pulling your shoulder blades down and back and keeping them in this position during your entire set will protect your shoulder joints and provide you with a solid base, enabling you to lift more. E: CONTINUE WITH FOOT POSITION."
            },
            "tempo": "2110 or 3010: 2 seconds eccentric phase (lowering the weight), 0-1 second at the bottom, 1 second for lifting (concentric phase). Why? The band helps most at the bottom of the lift, so a slightly slower eccentric (2-3 seconds) helps you control the weight and maximize muscle engagement throughout the range of motion. A brief pause at the bottom reinforces good form.",
            "commonMistakes": "1. Not keeping your shoulders down. By letting your shoulders shrug up towards your ears, you’ll make the movement less efficient, increase the stress on the shoulder joint, and could cause discomfort in the neck and shoulder area rather than exercising the targeted muscles. \n2. Half reps. Since the bar dip is quite a heavy exercise, it’s easy to start to cut off depth to be able to do more reps. Don’t do this. Focus instead on maintaining a full range of motion (in line with your mobility). If the movement is too heavy, add a resistance band to reduce some of the weight. \n3. Not maintaining form. Using momentum, swinging the body to complete the movement, arching your lower back, and/or flaring your elbows out to the sides, rather than relying on controlled, targeted muscle activation, is a common mistake while doing bar dips.",
            "comments": "The bench press against bands is a variation of the bench press in which resistance bands increase the tension as you press up. This makes it more challenging to lock out the lift, enhancing strength and power. It’s typically used to improve explosive strength and stability. Make sure to use the safety racks or to have a spotter while performing this exercise.",
            "alternativesAndVariations": "1. Bench Dip (puts more emphasis on your triceps, it's usually a bit easier). \n2. Bench Press: one of the most popular exercise, could be a good substitute if having shoulder pain when performing bar dips. \n3. Close-Grip Push-Up (more emphasis on your triceps)."
        }      
        
        2. HTML:
        """
        , cable_chest_press_html_text,
        """       
       
        2. Your response:
        "cable_chest_press": {
            "form": {
                "Steps": "1. Fasten a pair of handles in the top position of a cable cross. Grip the handles, step forward, and lean slightly forward. \n2. Push the handles forward until they meet in front of your body. \n3. With control, let the handles go back to the starting position.",
                "Form":" "
            },
            "tempo": "2010. Why? Cables provide constant tension, so a 2-second eccentric and smooth transition to the concentric maintain that tension for optimal muscle growth.",
            "commonMistakes": " ",
            "comments": "Cable chest press is a pressing exercise with a different resistance curve than for example dumbbell chest press, as the direction of resistance comes more from the sides. That might make it easier to keep tension on the chest muscles throughout the whole movement, and thus possibly benefit muscle growth.",
            "alternativesAndVariations": "Dumbbel chest press, Bench press and all its variations."
        }
        
        
        3. HTML:
        """
        , bench_press_html_text,
        """
        
        3. Your response:
        "bench_press": {
            "form": {
                "Steps": "1. Lie on the bench, pull your shoulder blades together and down, and slightly arch your back. \n2. Grip the bar slightly wider than shoulder-width apart. \n3. Inhale, hold your breath, and unrack the bar. \n4. Lower the bar with control, until it touches your chest somewhere close to your sternum. \n5. Push the bar up to the starting position while exhaling. \n6. Take another breath while in the top position, and repeat for reps.",
                "Form": "Position on the Bench. In most bench press federations, the back of your head, your upper back, and your glutes must remain in contact with the bench at all times, and your feet must touch the floor. You should place yourself close enough to the bar so that you can easily unrack it. If you are too close to the foot end of the bench, unracking the bar will be excessively heavy. If you are too close to the head end, you will hit the rack with the bar when you’re lifting. A good rule of thumb is to lie down so that your eyes or nose are directly below the racked bar. This will be close enough for you to unrack it easily, but not close enough to hit the rack while you’re benching. If you’re tall, you can probably slide even closer to the head end without hitting the rack, and if you’re short, you might have to be a little bit closer to the foot end. \nHand position: There are two major points to consider regarding how you grip the bar: 1. Your thumb must be on the opposing side of your other fingers, meaning that you grip around the bar. 2. The bar should rest in a comfortable place. As you’re lifting heavier and heavier weights, this might become tricky. For most people, the most comfortable and stable position is to hold the bar as close to the base of your palm as possible, i.e. close to or directly above your wrist. This minimizes the leverage the bar works on your wrist and makes it easier to hold. Combining the two points (keeping your thumb around the bar, while still keeping it low in your hand) above might be tricky, and one solution is to slightly rotate your hands inwards so that the barbell rests in a diagonal line in your palm. Don’t keep a completely straight wrist. This puts you at risk of losing balance, tilting your wrist forward, and dropping the barbell. Instead, tilt your wrist back slightly, to better rest the barbell in your hand. \nShoulder Blades: Your shoulders are mobile joints. But when you’re bench pressing, you want to trade some of that mobility for stability. Therefore, you will want to contract the scapulae (pull your shoulder blades down and back before you unrack the bar), and keep them in this position throughout the lift. If you don't know what this means, visualize that you’re putting them in your back pockets, but on the opposing side, i.e. your right shoulder blade into your left back pocket, and vice versa. Note that you will have to “set” your shoulder blades in this manner before you even unrack the bar, because when you’ve got a heavy bar in your hands, it’s too late. Then, your shoulder blades stay in this position until you re-rack the barbell again. That means you will keep your shoulder blades retracted and pulled down even as you’re pressing the bar up, which might require some practice. Pulling your shoulder blades down and back and keeping them in this position during your entire set will protect your shoulder joints and provide you with a solid base, enabling you to lift more. \nFoot position: The goal of your foot position is to keep you stable and support you throughout the lift. You will want to place your feet firmly on the floor and keep them still throughout the lift. Don’t fidget with your feet, as that will rob you of any stability. Don’t move your feet around after you’ve unracked the bar. Plant them, unrack, and then keep your feet still until you’ve racked the bar again. An exception is if you go up on your toes when you’re unracking, but in that case, plant your feet on the floor again as soon as you’re in position. \nLeg Drive (pro): Leg drive simply means that you apply a slight push with your legs. This will contract and tense your leg muscles, which in turn makes your whole body a little harder and more stable. And as the bench press is a lift that thrives on stability, you will likely be able to lift a little more weight because of this. To use leg drive in the bench press, push with your legs like if you tried to slide yourself over the head end of the bench. Meaning, you should apply force horizontally with your feet, away from the foot end of the bench. Note that you should only apply just enough force so that your feet don’t move. Your feet should still be completely still throughout the entire lift. You’re only pushing just enough to stay within the limits of friction, in order to tense your legs and stabilize your body."
            },
            "tempo": "2010 or 3010. Why? A controlled eccentric is key for proper form and muscle activation. You can experiment with a 2- or 3-second eccentric based on your strength and experience.",
            "commonMistakes":"1. Use a thumbless grip (this is having the thumb in the same side than all fingers). Firstly, it is not allowed in powerlifting. More importantly, it is also known as the suicide grip. This is because you can easily slip and drop the bar with a high risk of injury. Don’t use it. \n2. Round shoulders instead of contracting the scapulae (right position is with shoulder blades down). \n3. Push your feet down into the floor when using leg drive. This will only lead to you lifting your butt off the bench, which is, once again, not allowed in competition and not a recommended technique. ",
            "comments": {
                "Popularity":"The barbell bench press is one of the most popular exercises in the world and one of the best chest exercises you can do",
                "How Wide Should You Grip the Bar?": "Your correct grip width will depend on your anatomy, and also your goal with the exercise. Typically, people are stronger (can lift more weight) with a wider grip than with a narrower grip. The wider you grip the bar, the greater is the contribution of your chest and front delts. Conversely, the narrower you grip the bar, the greater is the contribution of your triceps. In a wide-grip bench press, your chest and front delts contribute 78% of the force necessary to lift the bar, while your triceps contribute 22%. In a close-grip bench press, the same numbers are 63% vs. 37%. The maximum grip width allowed in powerlifting is 81 cm between your hands. A powerlifting bar has rings marking this distance, and so your maximum grip width is when your index fingers cover these rings.",
                "Elbow Position: Where should your elbows be during the lift, close to your body or out to the sides?": "It depends on your grip width, your anatomy, and also your purpose with bench pressing. Grip width generally determines elbow position, with a wide grip typically leading to your elbows pointing out, and a close grip leading to your elbows being closer to your sides. If you bench press in order to grow your chest muscles, a rather wide elbow position and a wide grip is effective. If you bench press in order to train your triceps, a close grip with your elbows close to your sides is effective. If you want to lift as much weight as possible, something in between (around 45° to your sides) is probably best. You will, however, have to experiment for yourself and see what feels the best and suits your body",
                "Pro tip on Foot position": "Place your feet somewhere where you feel comfortable, stable, where you are able to produce some manner of leg drive. If you don’t know where to start, try placing your feet about hip-width apart, with your legs bent at a 90-degree angle or slightly more.",
                "Should You Pause With the Bar on Your Chest, or “Touch and Go”?": "If you don’t train for powerlifting, pausing in the bottom position is optional. It probably doesn’t have any additional benefits for building muscle compared to a controlled reversal. It might, however, help you improve your technique because you slow things down and get more practice in one of the hardest parts of the lift. Similarly, pausing can improve your strength off your chest. Bench pressing without a pause on your chest is called touch and go. You lower the bar down and touch your chest, without neither pausing nor bouncing, and then press the bar up again. Avoid bouncing the bar on your chest. It puts you at risk of injuring your ribs or your sternum, which can put a stop to your training for a long time.",
                "Pro tip:": "Push-ups work the same muscles as the bench press (= your chest, front deltoid, and triceps) in a similar movement, and can therefore be used as an bench press accessory exercise."
            },
            "alternativesAndVariations": "1. Close-Grip Bench Press, which puts more emphasis on your triceps. \n2. Feet-Up Bench Press, which greatly inhibits the amount of arch you can create with your lower back, and thus forces you into a more flat bench press technique. This leads to a longer range of motion, which, in turn, might benefit both your strength gains and muscle growth. \n3. Close-Grip Feet-Up Bench Press, combining Close-Grip Bench Press and Feet-Up Bench Press. Greater triceps emphasis, more training for your lockout, and stability training. \n4. Incline Bench Press. Inclining the bench emphasizes your upper chest muscles and anterior deltoids. Typically, a slight decline of about 30° is enough to get this effect. A popular variation of the incline bench press is to trade the barbell for a pair of dumbbells and perform incline dumbbell presses instead. Many people feel that the freedom of motion of the dumbbells enable them to lift in a more comfortable movement path. \n5. Smith Machine Bench Press. Doing bench press in a Smith Machine has pros and cons. On the pro side is the extra stability, which might enable you to work your muscles a little harder than usual because you don’t have to worry about balance. On the negative side, we have the fact that a normal, free barbell bench press doesn’t move in a vertical bar path (see the section on bar path for more info). The smith machine, however, forces you to work in a straight line, which an experienced bench presser might perceive as uncomfortable."
        }
        
        
        4. HTML:
        """
        , deadlift_html_text,
        """
        
        4. Your response:
        "deadlift": {
            "form": {
                "Steps": "1. Step up close to the bar so that it is about over the middle of your foot. \n2. Bend over and grip the bar with hands about shoulder-width apart. \n3. Inhale, brace your core slightly, and lift the bar by extending your hips and knees. \n4. Pull the bar close to your body, with a straight back, until you are standing straight up, with hips and knees locked out. \n5. Lower the bar back to the ground with control. \n6. Take another breath, and repeat for reps.",
                "Form": {
                    "1. Foot Position": "A rule of thumb is to point your toes in the same direction that your knee travels. This is not set in stone, however, and you still have some leeway to divert from this. You can choose to place your feet all the way together until your heels are touching. This will probably also require you to point your toes out a bit. More common and the best fit for most people, however, is to simply place the feet directly under your hip joints, with your toes pointing straight forward or only slightly out. What you’ll want to avoid is a stance so wide that your knees push your arms out. You’ll want your arms to be perpendicular to the floor as seen from the front, as this will result in the shortest and strongest range of motion. With a wide grip, you’d have to lean over more to reach the bar, resulting in a weaker starting position. It is a good idea to experiment a bit with different foot positions when you’re learning the deadlift, as your individual limb lengths will dictate what position suits you better.",
                    "2. Arms": "The best visualization of your arms’ roles in the deadlift is to act like ropes, attaching the bar to your shoulders. Preferrably, these should also be both long and straight ropes, as that will allow you to maximize your starting height in the lift. The first few inches of a deadlift is the biomechanically weakest part of the lift, and the more you can extend your knees and hips from the start, the easier it will be to pass this point. If you deadlift with your arms bent or excessively wide, you will have to lean over more, and thus end up in a weaker starting position.",
                    "3. Grip": "How to grip the bar? There are 4 common ways: \n1. Double overhand grip. This is the standard, intuitive way to grab a barbell. You simply hold on to it with both palms facing your body (pronated), and the thumbs opposing the other fingers. Pros: Comfortable and intuitive. The way you usually grab stuff. Symmetrical for your arms and shoulders. Cons: Weak! The barbell will want to roll out of your grip. This grip technique is usually fine for beginners learning the deadlift form, but quickly becomes a greatly limiting factor for how much you can lift. \n2. Hook grip. Common in weightlifting and  an evolution of the double overhand grip. Despite them being very similar, the hook grip increases the weights you can lift by 20–30% or more. You still grab the bar with a double overhand grip, with the difference that you stick your thumb under your index finger (and maybe also your middle finger). The thumb will act like a wedge or a chock and hinder the bar from rolling out of your grip. Pros: Much stronger than a regular overhand grip. Symmetrical for your arms and shoulders. Cons: Painful! It gets better with time, but will still be more uncomfortable than the normal way of gripping a bar. \n3. Mixed grip. Most common grip technique in powerlifting. Here, you counter the barbell’s rotation by supinating one hand – that is, turning one palm facing away from you. Like the hook grip, this greatly increases the weight you can lift. Pros: Much stronger than a regular overhand grip. Not very painful or wearing on your skin (compared to hook grip). Con: Asymmetrical for your arms and shoulders, which might transfer to an asymmetrical spine and hip position. It is unclear, however, if this actually is a problem. \n4. Lifting straps (not allowed in most powerlifting competitions). Pros: Using lifting straps will remove any grip related limitations for your pulls, will spare your skin, and also enable you to pull with a symmetrical overhand grip. Cons: 1. You won’t reap the benefit of developing your grip strength during training. 2. One more thing to carry around in your gym bag.",
                    "4. Hip position": "How high should your hips be in the starting position of a deadlift? Should you begin in more of a “squat” position, or something more along the lines of a stiff-legged deadlift? The answer can be deduced from three instructions: 1. Keep your arms vertical as seen from the side (or very close to vertical) during the entire lift. This is easy: once the bar gets heavy enough, it’ll be positioned right below your shoulder joint, or at least very close to it. 2. Keep the systems (your body + the bar = the system) center of mass roughly above your mid foot. How do you find this position? It’s the position in which you don’t lose your balance forward or backward during the pull – e.g. where you’ll automatically end up positioning the bar to keep your balance, without thinking about it. You can explore and reinforce this position by performing a pause deadlift, with the pause very low towards the floor. 3. Keep your shins as close to the bar as possible while still allowing for number 1 and 2 above, as that will enable you to more effectively use both your knee and hip extensors. For many people, this means having your shins touching the bar. These three points will have narrowed down your possible hip starting height quite a lot, but there is still some leeway regarding the exact height. The final adjustments of your hip position will depend on your personal strengths and weaknesses, especially in your back, hip and knee extensors. If you have a strong back and strong hip extensors but weak knee extensors, you’ll be the strongest (and naturally drift towards) when you are a little bit more bent over and thus shifting more of the load towards your hip. If you have strong knee extensors but a weaker back or weak hip extensors, you’ll be the strongest (and naturally drift towards) when you have a more vertical torso and your shins/knees closer to the bar (or even past the bar, for taller lifters). Note that your ideal starting position probably is one that you already know and use. When we perform movements, we usually experience a bit of micro-variation. The movement simply doesn’t look exactly the same each rep, and if we by chance discover a slightly easier and more efficient way of performing a rep, we usually take notice – either consciously or subconsciously. That is why, as long as you’re not doing something fundamentally wrong – like keeping the bar way ahead of your shins – your current starting position is probably also your most effective one. At least right now. As your body and musculature changes with continued training, so will your strengths and weaknesses change. That means that the most effective deadlift position in the beginning of your lifting career, might not be the same as your most effective lifting position when you are on your peak. Do experiment with a slightly higher or lower hip position in the start – although my bet is that you’ll find that you’re either weaker with the new position, or will end up in your old position anyway when the load gets heavy enough.",
                    "5. Back position & Bracing":"In general, you should deadlift with a neutral spine. Generally avoid lifting with a flexed back in your training, as it could increase your risk of injury – if you cannot maintain a neutral spine, lower the weight or perform less reps per set. To maintain a neutral spine, you can implement bracing, which refers to the act of keeping your core rigid during a lift. Start by taking a deep breath just before lifting, about 80% of maximum. Then, it’s time to tense up your core muscles. Effective deadlifting requires efficient force transfer. The force must be transferred from the bar, via your grip, arms and shoulders, through your torso down to your hips, and finally down via your legs and feet to the floor. A key to effective force transfer is stiffness, and this ties in with the previous section on bracing. Arms and legs won't heavily help here.  The core however, with it’s pelvis connected to the spine  is in great need of stabilization – especially in the conventional deadlift since it places greater demands on your low back. A braced and stiff core will be essential for effective force transfer. Before you push the pedal to the metal and revv up those strong prime movers in your back, hip and knee extensors, you’d better take a moment to make sure your core is good to go, and this is done by building tension. It is simply the act of consciously tensing your body to prepare it for what is about to come.",
                    "6. Lift-Off":"The lift-off is really about nailing down two things: 1. Knowing your strongest pulling technique, and 2. Using and maintaining that technique. Your own strongest pulling technique will entail things like: how you distribute work between hip and knee extensors, what sequence and speed you will extend different joints in, and whether or not you are going to pull with a rounded or straight back. The deadlift is more segmented into three distinct phases: 1. Mostly knee extension, 2. Mostly hip extension, 3. Simultaneous knee and hip extension until lock-out. Some generally  helpful cues for lift-off: Push the floor away. Instead of pulling the barbell up, you focus on pushing the floor away, or down. Chest up, this is one of the best ways to help a lifter maintain a more straight back.",
                    "7. Lock-Down":"Assuming you are strong enough to complete the lift, the lock-out part of a deadlift is  pretty simple. All you have to do is: extend your knees, hips and back, and keep your shoulder back for the lift to be considered completed. Simply drive your hips forward, kind of like a hip thrust. If you’re having troubles extending your spine in the top or having “soft knees” (not fully extended), a helpful visualization might be to imagine a chain attached to the top of your head, pulling you upwards.",
                    "8. Setting the Bar Down":"How to set the bar down safely. acktrack your steps from the pull, even though we compress it slightly into two distinct phases: 1. Hip hinge. Initiate the eccentric by pushing your butt back and leaning your torso forward. The knees bend, but not as fast as your hip. Lower the bar along your legs like this until you’ve cleared the knees. 2. Knee bend. When the bar has cleared your knees, you are probably leaning as far forward as you’re going to get. At this point you take out the rest of the movement with your knees, and simply bend them until the bar touches the ground."
                }
            },
            "tempo": "11X1: 1021, 1031, oe 1041: 1 second to lift the weight explosively, 1 second to pause at the top, and control the bar on the way down for X seconds (2-4 seconds depending on the weight and individual preference). Why? The explosive lifting phase (1 second) recruits the maximum number of muscle fibers and develops power. Controlling the weight on the way down (eccentric phase) helps maintain tension in the muscles, promotes hypertrophy, and reinforces good form. The brief pause at the top prevents loss of tension and ensures continuous muscle engagement. The variable tempo (X) on the eccentric phase allows for individual adjustment based on experience level and training goals. Beginners may benefit from a slower descent (4 seconds) to focus on technique and control, while experienced lifters may use a faster tempo (2 seconds) to increase training volume and intensity.",
            "commonMistakes": {
                "Feet & Legs stance - Have  the stance is so wide that the knees push the elbows out":"This forces you to lean forward, thus bringing you closer to the floor, and into a biomechanically weaker position. Avoid this if possible. It may be, however, that your thighs and abdomen are too big for a narrow stance, and thus forcing you to adopt a stance similar to this.",
                "Arms - Bend the arms or grip the barbell excesively wide":"This will make you have to lean over more, and thus end up in a weaker starting position, making it harder to start the movement and increasing the injury risk.",
                "Hip position":"[E - Leave only if image present] Here (image) I am trying to show different hip positions. My anatomy doesn’t change between the pics, so only one of these three are really a viable option for me. The left pic with a high hip position is just fake posing: note the angle of the arm. As soon as I would try to lift a heavy weight that way, my arm would swing out to an angle perpendicular to the direction of gravity, and the moment arm from the weight to my hips would be very long and ineffective for heavy pulls. In the right pic with a low hip position on the other hand, the knees would be in the way of the bar. My personal starting position is the middle one. Just right for me."
            },
            "comments": {
                "Conventional or Sumo Deadlift?": "\n",
                "Discover your ideal Lift-Off position":"You will have to experiment to find out for yourself: Are you stronger when you let your butt shoot up in the start, or when your patiently push with your legs until the bar finally gives? Are you stronger when you deadlift with a straight back during the whole lift, or when you initally round your back but re-extend it during the lock-out? Your individual optimal deadlift technique will vary, but if we yet again shall revert to generalizations, we believe that most lifters will eventually have their greatest performances when they patiently let their quadriceps contribute to the lift, resulting in a pronounced knee extension in the start, as well as pulling with a more or less straight back through the first phase of the lift, thereby ridding themselves the task of re-extending their spine. So, how do you go about accomplishing this? Both depend on grooving the right neuromuscular motor patterns, which in plain English means train a lot. With the desired technique. With progressively heavier weights, as you get increasingly good at maintaining your technique with heavier weights. To train this, you will be needing some kind of feedback to discern whether you’re doing it right or wrong: A coach/friend/GymAlly who knows what to look for might give you feedback after a set or even after every rep.",
                "Strength advantage to controlled Eccentric":"Is there a strength advantage to setting the bar down slowly in your training (controlled Eccentric)? The tricky part of a deadlift is the way up – the concentric phase. That is what you compete with in powerlifting, and the way down (the eccentric phase) is mostly a formality, where the only rule is to lower the bar with control and maintaining your grip on it until it rests on the floor. However, for general training, it's good to know that several studies has compared the concentric strength gains from training protocols where you either perform concentric reps only or concentric+eccentric (that is, normal training) reps, and found that concentric+eccentric training yields greater concentric strength gains. This means that even if you perform twice as many reps where you set the bar down quickly (without really braking it’s fall), you’re still not getting as good gains as if you were to really slow down the bar, and lower it under, say, at least two seconds. Thus, a little bit more work and discomfort invested in your deadlift training in the shape of more prolonged eccentrics might pay good dividends.",
                "How to Become Stronger in the Deadlift?":"The weight you can lift in the deadlift is determined by the amount of force you can produce in your muscles (primarily the extensors of the hip, knee, and back) and transfer into the bar. To deadlift more weight, you must generate more force into the bar. There are three different domains in which you can improve in order to accomplish this: \n1. Muscle & tendon. A bigger muscle produces more force, and a stiffer tendon transfers that force into your bones more efficiently. Skeletal muscle mass is strongly correlated with deadlift performance. To build bigger muscles, you need to train them in any reasonable way. You can train them deadlifting, but also doing other exercises. E.g. If you build bigger and stronger glutes using hip thrusts, you will still have a bigger and stronger hip extensor when you go back to the deadlift. Tendons increase in stiffness from all heavy training. Improvements in neuromuscular efficiency and especially technique require more specificity, and you will actually have to practice the lift in which you want to improve. Preferrably at pretty heavy loads.",
                "Training Volume for the Deadlift":"The optimal training volume varies widely between individuals and is dependent on several factors, such as training status, age, nutrition (especially calories and protein ingestion), and recovery. The relationship between strength gains and training volume follows an inverted U-shape, where both too little and too much training leads to inferior gains. The key is to find the Optimal training volume. You’ll neither want to train too little nor too much for optimal gains. But if you have to choose, erring on the side of too little will at least spare you time and decrease your injury risk. Some general volume recommendations in terms of working sets of deadlifts. That is sets that are challenging but are generally not performed to failure but leaving 1–2 reps in the tank. \n1. Low volume, 3–5 sets per week: This is a suitable volume for the beginner lifter that is simultaneously getting used to squatting maybe 1–2 times a week and thus is acclimatizing to a lot of new stress. It might also be suitable for the intermediate lifter, either tapering or taking a minimalistic approach to deadlifting while perhaps focusing on hypertrophy work for the deadlifting musculature. \n2. Medium volume, 6–13 sets per week: The goldilocks zone, and probably a nice target volume for most people in the intermediate stage. Recovery demands will be quite low if this is the only training you do for the lower body, but if you add in squats and deadlift accessory training, the total volume will place quite high demands on your recovery. This might be a suitable volume for an advanced lifter who is tapering. \n3. High volume, 14–20 sets per week: For the intermediate lifter who is stuck on a plateau and wants his or her deadlift to ascend to the next level, or for the advanced lifter. Recovery demands are high, especially if this is coupled with squat training and accessory work. Most will have to gradually increase their volume to this level over a long time, and some lifters might find it too much for them, and that medium volume works better.",
                "Training for Muscles Used in the Deadlift":"For optimal deadlift gains, you will probably want to train the prime mover muscles more than you can do if you’d only use deadlifts, by doing assistance exercises. By varying your exercises, you can probably lower the risk of over-use injuries. Assistance exercises are usually for hypertrophy, in this case, exercises that strengthen and grow the muscles used in the lift. Major muscles worked in the deadlift: Knee extensors (Quadriceps), Hip extensors (Glutes, Hamstrings, Adductors), and Back extensors (Lower back). The options for hypertrophy exercises are many and more a matter of personal preference. However, here is a sample list of exercises that you might use to supplement your training for the muscles involved in the deadlift. If you are hitting squats and deadlifts hard, you might only want to pick 1–2 of the exercises below and do 2–3 sets x 1–2 workouts per week. If you deadlift with a low volume (relative to your training level), you can increase the volume of assistance exercises. Sample exercises: Squat (if you’re not already doing them) (knee, hip and back extensors), Bulgarian split squat (knee and hip extensors), Hip thrust (hip extensors), Kettlebell swing (hip and back extensors), Hyperextension (hip and back extensors), Good morning (hip and back extensors), Romanian deadlift (hip and back extensors), Barbell row (upper back and isometric training for back and hip extensors), Farmers walk (core, grip, trapezius, and stability training for hips)",
                "Training Frequency for the Deadlift":"Training frequency might be an important mediator for training volume. Do you want to train a lot? Well, sooner or later you’ll have to distribute the training on more days, before your current workouts swell beyond the scope of what is productive to do in one session. A rough limit on the number of work sets of deadlifts in a single session might be somewhere in the neighbourhood of 8-10 sets for an experienced lifter. About 1–2 deadlift sessions per week is not a bad starting point for someone who also trains squats or similar lower-body movements. Most lifters can probably do one hard deadlift session per week, while two sessions per week might require most lifters to dial down the intensity a bit. A classic recommendations for adapting to a larger workload that you might use for the deadlift is this: \n1. First increase the frequency. Go from one deadlift session to two, more or less by splitting your current training volume in two. \n 2. Then increase the volume. Slowly ramp up the volume by adding easy sets, so that you are well on the safe side of doing too much, too soon. \n 3. Finally increase the intensity (load). When you are accustomed to two deadlift sessions per week, you might start to increase the intensity of load you’re using.",
                "Load & Reps for the Deadlift":"Your strength can improve from training with both high and low loads, but when you compare them head to head, lifting heavy weights is slightly more effective if you want to get better at lifting heavy weights. There are several rep and loading strategies that work, as long as you heed the principle of specificity (i.e. lifting heavy every once in a while) and progressive overload (e.g. adding weight or reps). One basic template you could try is to Slug away at 75-80% of 1RM. That range is heavy enough to be specific training for max attempts, but not so heavy that it burns you out. It’s typically possible to accumulate a large, effective volume at this intensity. Something like 6-14 sets x 2–3 reps weekly might be appropriate for an intermediate level practitioner, divided in 1-2 sessions, and with an expected increase of 2.5kg per workout."
            },
            "alternativesAndVariations": {
                "Hypertrophy: Exercises that will strengthen and grow the muscles used in the lift.": "The primary movers in the deadlift are the knee extensors (quadriceps), hip extensors (glutes, hamstrings, adductors), and back extensors (mostly your lower back), so that is the muscles you should be most concerned about strengthening and growing. The options for hypertrophy exercises are many, and more a matter of personal preference than anything else. We will now share some exercises that you might use to supplement your training for the muscles involved in the squat. If you are not sure, we recommend to pick squats + 1–2 of the exercises below, and do 2–3 sets at 1–2 workouts per week. If you squat with a low volume (relative to your training level), you can increase the volume of supplementary exercises: 1. Squat (knee, hip and back extensors), 2. Bulgarian split squat (knee and hip extensors), 3. Hip thrust (hip extensors), 4. Kettlebell swing (hip and back extensors), 5. Hyperextension (hip and back extensors), 6. Good morning (hip and back extensors), 7. Romanian deadlift (hip and back extensors), 8. Barbell row (upper back and isometric training for back and hip extensors), 9. Farmers walk (core, grip, trapezius, and stability training for hips)",
                "Technique: Exercises that strengthen the movement, or improve the performance or technique of the lift.":"These are generally more taxing than the hypertrophy exercises listed earlier, and also cut in more into your other deadlift volume. Working sets of the exercises listed below should probably be traded in for your regular sets of deadlift if you’re already training at a high volume, rather than piled on top of it. \n1. Pause deadlifts. Can be used to improve overall technique and strength around the position that you pause in. We generally recommend pausing as close to the floor as possible (just 3-5cm or so above), and for a count of three. \n2. Deficit deadlifts. Can be used to challenge and strengthen the start in the deadlift, and also might bring about more training for your legs. Do these by standing on a weight plate or some sturdy block, around 1–4 inches high, so that you’ll have to reach down further to grab and lift the bar. \n3. Rack pulls. Useful for getting used to heavier weights in your hands, and overloading the later part of a deadlift. Choose rack height at your own discretion, but a common height is to begin with the bar just below your knees. \n4. Trap bar deadlifts. Distributes the work more evenly between hip and knee extensors, and unloads the back a bit. Can be useful for more balanced development of the lower body, and is a back-friendly substitute."
            }    
        }
        
        
        5. HTML:
        """
        , ex_html_text,
         
        """  
        Important: IGNORE ALL URLs.
              
        5. Your response:
        """],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    exercise_json_from_html = []  # Use a list to store chunks
    for response in responses:
        exercise_json_from_html.append(response.text)  # Append each chunk

    exercise_json_from_html = "".join(exercise_json_from_html)  # Join the chunks
    
    return exercise_json_from_html  # Return the JSON

In [15]:
print(ex_json)

{"hip_thrust": {"form": {"Steps": "1. Sit on the floor with your back against a sturdy bench., 2. Roll the barbell up over your thighs, until it is placed over your hips., 3. Place your feet on the floor, about shoulder-width apart, with bent knees., 4. Place your hands on the bar to stabilize it., 5. Push the bar towards the ceiling by extending your hips. Your knees should form a ~90 degree angle at the top., 6. Lower the weight and repeat for reps.", "Bench": "We recommend placing something between the bar and your pelvis to soften the pressure from the bar, like a folded yoga mat, a towel, or a barbell pad."}, "tempo": "2011: 2 seconds eccentric phase (lowering the weight), 0 seconds at the bottom, 1 second for lifting (concentric phase), and 1 second pause at the top. Why? A controlled eccentric phase maximizes muscle engagement and a brief pause at the top ensures peak contraction of the glutes.", "commonMistakes": null, "comments": "The hip thrust is an isolation exercise for yo

In [6]:
# Correct the firstly generated JSON - Few-shot learning

def gen_concise_json(ex_json):
    vertexai.init(project="testing-elena", location="europe-west4")
    model = GenerativeModel(gemini_latest_stable)
    responses = model.generate_content(
    [f"""
    You are a generator of valid, detailed, and complete JSON files. You will start by an input JSON, which includes different sections useful to know how to perform a specific strength exercises.
    The section structure (form, tempo, commonMistakes, comments, and alternativesAndVariations) should remain the same. However, you will need to make some changes to the JSON.
    Aspects you will need to review and fix in input JSON:
    - Delete information on powerlifting, as this JSON will NOT be for powerlifters, but for intermediate strength practitioners. Just include information on powerlifting or powerlifters if it's as a fun fact (comments section) that could be useful for the average gym user.
    - Delete unnecessary verbose sentences and words. This JSON should include only useful information, as it will be input for another system, not red by humans.
    - In the same note, delete or condense sentences that mean the same thing. Don't repeat information.
    - Place JSON line breaks (\n) when needed, e.g. to separate steps or paragraphs in one same section or item inside the JSON file, e.g. steps in an exercise, or different variations, or different aspects that could be clearer in a numbered list with line breaks.
    - Fix JSON structure if needed (to generate a valid JSON)
    - Review that each key-value pair information is in its proper section, otherwise place them (or a subset of it as a new key-value pair) in the correct section. Use the following sections purpose for this:
        1. Form: Information that indicates how to perform correctly the exercise, from a form POV. Not fun facts, nor comments or alternatives, just the info on the correct technique to perform this exercise (objectively). Should be split into "Steps" and "Form" subsections. This will be used to review that user is performing the exercise correctly.
        2. Tempo: Describes the speed of each phase of a repetition. It's usually written as a four-digit number (e.g., 2010). Here's what each digit represents:
            - 1st digit: Eccentric phase (lowering the weight)
            - 2nd digit: Pause at the bottom of the movement
            - 3rd digit: Concentric phase (lifting the weight)
            - 4th digit: Pause at the top of the movement
            If empty, complete this section based on your own knowledge.
        3. Common mistakes: Aspects that users usally fail or miss when performing this exercise. This should include info that we should be attentive to in order to correct the users technique, in case they are doing some of these mistakes.
        4. Comments: Comments and "Pro tips". Basically anything that is not "the one an only way to perform this exercise with the right form", but could be useful information for the user performing this exercise. Anything that can expand or enhance their knowledge on the exercise.
        5. Alternatives and Variations: Info on any kind of auxiliary or complementary exercises for the one described.
    - Don't add any extra information on the JSON (only tempo if not present).
    - IMPORTANT: MAINTAIN ALL information that could be useful (DO NOT SUMMARIZE the input JSON).
    
    ---
    
   Example 1 - Input JSON:
    """
    , read_json_as_string('data/jsons/for_llm_calls_only/old_squat.json'),
    """
   
   Example 1 - Output JSON (REMEMBER TO MAINTAIN ALL information that could be useful (DO NOT SUMMARIZE THE INPUT JSON):
    """
    , read_json_as_string('data/jsons/air-squat.json'), 
    """
    ---
    
   Example 2 - Input JSON:
    """
    , read_json_as_string('data/jsons/for_llm_calls_only/deadlift_gen1.json'),
    """
   
   Example 2 - Output JSON (REMEMBER TO MAINTAIN ALL information that could be useful (DO NOT SUMMARIZE THE INPUT JSON):
    """
    , read_json_as_string('data/jsons/deadlift.json'), 
    """
     ---
    
    Your Turn - Input JSON:
    """
    , ex_json,
    
    """
    Your Turn - Output JSON REMEMBER TO MAINTAIN ALL information that could be useful (DO NOT SUMMARIZE THE INPUT JSON). Your response:
    """],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    concise_json = []  # Use a list to store chunks
    for response in responses:
        concise_json.append(response.text)  # Append each chunk

    concise_json = "".join(concise_json)  # Join the chunks
    
    return concise_json  # Return the JSON

In [17]:
# Output result from Call 2
concise_json = gen_concise_json(ex_json)

In [18]:
print(concise_json)

{
"hip_thrust": {
"form": {
"Steps": "\n1. Sit on the floor with your back against a sturdy bench. \n2. Roll the barbell up over your thighs, until it is placed over your hips. \n3. Place your feet on the floor, about shoulder-width apart, with bent knees. \n4. Place your hands on the bar to stabilize it. \n5. Push the bar towards the ceiling by extending your hips. Your knees should form a ~90 degree angle at the top. \n6. Lower the weight and repeat for reps.",
"Form": {
"Bench": "We recommend placing something between the bar and your pelvis to soften the pressure from the bar, like a folded yoga mat, a towel, or a barbell pad."
}
},
"tempo": "2011: 2 seconds eccentric phase (lowering the weight), 0 seconds at the bottom, 1 second for lifting (concentric phase), and 1 second pause at the top. Why? A controlled eccentric phase maximizes muscle engagement and a brief pause at the top ensures peak contraction of the glutes.",
"commonMistakes": {
"1. Raising hips too high": "Raising you

### Iterate over all non-complete exercises (the most typical ones)
1. Take all JSONs from original scrapper where "commentary" is "Text and graphics from the \nStrengthLog \napp." or commentary is "" (empty).
2. Manually download their HTMLs.
3. Iterate on them with the scrapper enhancer.


In [13]:
# From the JSON file 'strengthlog_results--2024-06-27--17-16', take all the "name" key-value pairs where the fields where "commentary" is "Text and graphics from the \nStrengthLog \napp." or commentary is "" (empty). 
def extract_non_complete_exercises_from_json():
    # Initialize an empty list to store the names
    names = []

    file_path = 'strengthlog_results--2024-06-27--17-16.json'
    
    # Open and load the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Iterate through each item in the JSON data
    for item in data:
        # Check if the commentary is as specified
        if item.get('commentary') in ["Text and graphics from the \nStrengthLog \napp.", ""]:
            # If condition is met, add the name to the list
            names.append(item.get('name'))

    print(names)
    print(len(names))
    
    return names

# Example usage

names = extract_non_complete_exercises_from_json()

['dumbbell-chest-fly', 'dumbbell-incline-press', 'machine-chest-fly', 'standing-cable-chest-fly', 'standing-resistance-band-chest-fly', 'barbell-upright-row', 'dumbbell-lateral-raise', 'face-pull', 'overhead-press', 'reverse-cable-fly', 'reverse-dumbbell-flyes', 'seated-dumbbell-shoulder-press', 'barbell-curl', 'barbell-preacher-curl', 'dumbbell-curl', 'hammer-curl', 'barbell-standing-triceps-extension', 'barbell-lying-triceps-extension', 'overhead-cable-triceps-extension', 'tricep-pushdown-with-bar', 'tricep-pushdown-with-rope', 'bulgarian-split-squat', 'goblet-squat', 'leg-extension', 'leg-press', 'pause-squat', 'seated-leg-curl', 'squat', 'zercher-squat', 'back-extension', 'banded-muscle-up', 'barbell-row', 'deadlift', 'dumbbell-shrug', 'jumping-muscle-up', 'lat-pulldown-with-pronated-grip', 'bar-muscle-up', 'ring-muscle-up', 'pause-deadlift', 'rack-pull', 'rowing-machine', 'stationary-bike']
42


In [13]:
# Removing the ones we already processed
names.remove('deadlift') # Removed deadlift, dumbbell-curl, and squat (from previous tests), and dumbbell-chest-fly (from mini-test before iterating)
names.remove('squat')
print(len(names))

18


In [11]:
# # Remove next 14 exercises
# for exercise_code in names[:7]:
#     names.remove(exercise_code)

# print(names)

['goblet-squat', 'leg-extension', 'leg-press', 'pause-squat', 'seated-leg-curl', 'squat', 'zercher-squat', 'back-extension', 'banded-muscle-up', 'barbell-row', 'deadlift', 'dumbbell-shrug', 'jumping-muscle-up', 'lat-pulldown-with-pronated-grip', 'bar-muscle-up', 'ring-muscle-up', 'pause-deadlift', 'rack-pull', 'rowing-machine', 'stationary-bike']


In [17]:
def process_exercise_codes(names):
    """
    Process the first 10 exercise codes by generating a concise JSON from their HTML representation
    and saving it to a specific directory. Then, remove these processed codes from the list.
    
    Args:
    - names: A list of exercise codes.
    """
    # Process only the first 10 exercise codes
    for exercise_code in names[:10]:
        # Step 1: Generate HTML text from the exercise code file
        ex_html_text = html_string(f'{exercise_code}.html')
        
        # Step 2: Generate JSON from the HTML text
        ex_json = gen_json_from_html(ex_html_text)
        
        # Step 3: Generate a concise version of the JSON
        concise_json = gen_concise_json(ex_json)
        
        # Step 4: Save the concise JSON to a file in the specified directory
        output_path = os.path.join('data', 'jsons', f'{exercise_code}.json')
        with open(output_path, 'w') as json_file:
            json_file.write(concise_json)
            
        print(f'JSON Generated: {exercise_code}.json')
    
    # Remove the first 10 processed codes from the list
    del names[:10]

In [ ]:
process_exercise_codes(names)

## V3 - Start from JSON (for those exercises that are complete)

#### Helper function 
To retrieve JSON content from scrapper given an input exercise

In [67]:
def get_exercise_content(exercise_name):
    """
    Load a JSON file and return the content associated with a given exercise name.
    
    :param exercise_name: Name of the exercise to search for.
    :return: Dictionary with the exercise content or None if not found.
    """
    file_path = 'strengthlog_results--2024-06-27--17-16.json'
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            for exercise in data:  # Assuming the JSON structure is a list of exercises
                if exercise.get('name') == exercise_name:
                    return exercise
    except FileNotFoundError:
        print(f"File {file_path} not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file {file_path}.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    
    return None

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
    "response_mime_type":"application/json", # Force the model to return a JSON. Note: Not always a valid one!
}

In [37]:
# Test
exercise_content = get_exercise_content('barbell-lunge')
if exercise_content:
    print(json.dumps(exercise_content, indent=4))
else:
    print("Exercise not found.")

{
    "id": 94,
    "name": "barbell-lunge",
    "musclesInvolved": {
        "image": "https://i0.wp.com/www.strengthlog.com/wp-content/uploads/2020/10/Muscles-worked-by-barbell-lunge.png?resize=700%2C700&ssl=1",
        "primary": [
            {
                "label": "Quads",
                "link": "https://www.strengthlog.com/quad-muscles-exercises-workout/"
            },
            {
                "label": "Glutes",
                "link": "https://www.strengthlog.com/glute-muscles-exercises-workout/"
            },
            {
                "label": "Adductors",
                "link": "https://www.strengthlog.com/hip-adductor-muscles-exercises-workout/"
            }
        ],
        "secondary": []
    },
    "executionSteps": [
        "Take a big step forward and sink as deep as possible in a lunge position, without hitting the knee of the back leg in the floor.",
        "Return to the starting position by pushing yourself back with the front leg."
    ],
    "

## Generate new JSON for non-worked exercises

### Test with one exercise

In [65]:
# Generate exercise JSON by correct the JSON generated by the scrapper - Few-shot learning

def gen_json_based_on_scrapper(ex_json_name):
    vertexai.init(project="testing-elena", location="europe-west4")
    model = GenerativeModel(gemini_latest_stable)
    responses = model.generate_content(
        [f""" 
        You are a generator of valid, detailed, and complete JSON files. You will start by an input JSON, which includes different sections useful to know how to perform a specific strength exercises.
        However, the input JSON is not very well structured, you will need to restructure it and use your knowledge on  to expand its content.
        The new section structure will be:
            1. form. Two sub-sections:
            1.1. Steps: Describes step by step how to perform an exercise.
            1.2. Form: Describes with the best detail possible all the movements involved in the exercise in order to perform it with the proper form. Usually has separate sub-sections on its own, one for each different aspect to explain on the form in a different key-value pair.
            
            2. commonMistakes: Contains typical mistakes that users make when performing this exercise.
            
            3. comments: This section can contain as many sub-sections as interesting topics can be commented on this exercise. Separate each comment in a different key-value pair. Some examples could be:
            - Different ways or positions to perform the exercise, and which muscles get more involved in each.
            - Pro tips of any kind
            - Basically any information that can be valuable for the user and is not covered in any other section.
            
            4. alternativesAndVariations: Other exercises that are variations of this (e.g. with different material) or other exercises train similar muscle groups, together with the main difference with this one.
            
            Then, basen on your own knowledge, complete the last section:
            5. tempo. Tempo describes the speed of each phase of a repetition. It's usually written as a four-digit number (e.g., 2010). Here's what each digit represents:
                - 1st digit: Eccentric phase (lowering the weight)
                - 2nd digit: Pause at the bottom of the movement
                - 3rd digit: Concentric phase (lifting the weight)
                - 4th digit: Pause at the top of the movement
            In this section, you need to set the ideal tempo (could be a range) to perform this exercise and describe it (the Why), based on your own knowledge.
            
        Once you have restructured the JSON, you will also need to make some changes to it.
        Aspects you will need to review and fix in input JSON:
        - Delete information on powerlifting, as this JSON will NOT be for powerlifters, but for intermediate strength practitioners. Just include information on powerlifting or powerlifters if it's as a fun fact (comments section) that could be useful for the average gym user.
        - Delete unnecessary verbose sentences and words. This JSON should include only useful information, as it will be input for another system, not red by humans.
        - In the same note, delete or condense sentences that mean the same thing. Don't repeat information.
        - Place JSON line breaks (\n) when needed, e.g. to separate steps or paragraphs in one same section or item inside the JSON file, e.g. steps in an exercise, or different variations, or different aspects that could be clearer in a numbered list with line breaks.
        - Fix JSON structure if needed (to generate a valid JSON)
        - Review that each key-value pair information is in its proper section, otherwise place them (or a subset of it as a new key-value pair) in the correct section. Use the following sections purpose for this:
            1. Form: Information that indicates how to perform correctly the exercise, from a form POV. Not fun facts, nor comments or alternatives, just the info on the correct technique to perform this exercise (objectively). Should be split into "Steps" and "Form" subsections. This will be used to review that user is performing the exercise correctly.
            2. Tempo: Describes the speed of each phase of a repetition. It's usually written as a four-digit number (e.g., 2010). Here's what each digit represents:
                - 1st digit: Eccentric phase (lowering the weight)
                - 2nd digit: Pause at the bottom of the movement
                - 3rd digit: Concentric phase (lifting the weight)
                - 4th digit: Pause at the top of the movement
                If empty, complete this section based on your own knowledge.
            3. Common mistakes: Aspects that users usally fail or miss when performing this exercise. This should include info that we should be attentive to in order to correct the users technique, in case they are doing some of these mistakes.
            4. Comments: Comments and "Pro tips". Basically anything that is not "the one an only way to perform this exercise with the right form", but could be useful information for the user performing this exercise. Anything that can expand or enhance their knowledge on the exercise.
            5. Alternatives and Variations: Info on any kind of auxiliary or complementary exercises for the one described.

        ---

        Example 1 - Input JSON:
        """
        , json.dumps(get_exercise_content('barbell-lunge')),
        """

        Example 1 - Output JSON:
        """
        , read_json_as_string('data/jsons/barbell-lunge.json'), 

        """


        Example 2 - Input JSON:
        """
        , json.dumps(get_exercise_content('close-grip-bench-press')),
        """

        Example 2 - Output JSON:
        """
        , read_json_as_string('data/jsons/close-grip-bench-press.json'), 

        """


        Your Turn - Input JSON:
        """
        , json.dumps(get_exercise_content(ex_json_name)),
        """

        Your Turn - Output JSON:
        """],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    output_json = []  # Use a list to store chunks
    for response in responses:
        output_json.append(response.text)  # Append each chunk

    output_json = "".join(output_json)  # Join the chunks
    
    return output_json  # Return the JSON

In [43]:
gen_json = gen_json_based_on_scrapper('close-grip-bench-press')
print(gen_json)

{"close-grip-bench-press": {"form": {"steps": "\n1. Lie on the bench with your feet flat on the floor, pulling your shoulder blades together and down, and slightly arch your back. \n2. Grip the bar with an overhand grip, narrower than in a regular bench press, so that your hands are slightly closer than shoulder-width apart. \n3. Take a breath and hold it, and unrack the bar by straightening your arms. \n4. Lower the bar with control until it touches your chest somewhere around the nipple line, keeping your elbows tucked in at about a 45-degree angle. \n5. Push the bar up to the starting position while exhaling, keeping your elbows slightly bent. \n6. Take another breath in the top position, and repeat for reps.", "form": {"Grip": "Overhand grip, slightly closer than shoulder-width apart.", "Elbows": "Keep your elbows tucked in at about a 45-degree angle during the movement.", "Back": "Maintain a slight arch in your back, but keep your butt on the bench.", "Shoulder Blades": "Keep your

### Iterate over all exercises not processed yet
#### Helper functions

In [2]:
# Create list for the names that have already been produced
def list_registered_exercises():
    """
    List all exercises in the specified directory, excluding the '.json' extension from the names.

    :param directory: Path to the directory containing the exercise JSON files.
    :return: A list of exercise names without the '.json' extension.
    """
    # List all files in the directory
    files = os.listdir('data/jsons/')
    
    # Filter out files that are not JSON
    json_files = [file for file in files if file.endswith('.json')]
    
    # Remove the '.json' extension from each file name
    exercise_names = [os.path.splitext(file)[0] for file in json_files]

    print(exercise_names)
    print(len(exercise_names))
    
    return exercise_names


# Create a list of all available exercises in the scrapper ('strengthlog_results--2024-06-27--17-16' file). From the JSON file , take all the "name" values
def list_all_exercises_from_scrapper_json():
    # Initialize an empty list to store the names
    names = []
    
    # Open and load the JSON file
    with open('strengthlog_results--2024-06-27--17-16.json', 'r') as file:
        data = json.load(file)
    
    # Iterate through each item in the JSON data
    for item in data:
        names.append(item.get('name'))
    
    print(names)
    print(len(names))

    return names

#### 1. Exercises not retrieved through normal scrapper
Before iterating over these exercises, we need to understand which are remaining to be processed

In [48]:
# Create list for the names that have already been produced
registered_exercises = list_registered_exercises()

['smith-machine-squat', 'body-weight-lunge', 'one-handed-cable-row', 'snatch-grip-deadlift', 'dumbbell-deadlift', 'chin-up', 'triceps-bodyweight-extension', 'floor-press', 'dumbbell-wrist-curl', 'machine-chest-fly', 'dumbbell-lying-triceps-extension', 'lying-windshield-wiper', 'dumbbell-decline-chest-press', 'standing-glute-kickback-in-machine', 'reverse-hyperextension', 'good-morning', 'lateral-walk-with-band', 'hammer-curl', 'seated-dumbbell-shoulder-press', 'bulgarian-split-squat', 'block-clean', 'cable-close-grip-seated-row', 'barbell-walking-lunges', 'dumbbell-horizontal-external-shoulder-rotation', 'bodyweight-curl', 'bar-muscle-up', 'stationary-bike', 'seated-smith-machine-shoulder-press', 'push-press', 'barbell-hack-squat', 'box-squat', 'board-press', 'barbell-wrist-extension', 'band-external-shoulder-rotation', 'barbell-standing-calf-raise', 'tricep-pushdown-with-bar', 'deficit-deadlift', 'reverse-barbell-lunge', 'leg-press', 'dumbbell-romanian-deadlift', 'kneeling-incline-pus

Let's first work on the exercises that were not complete in the scrapper.

In [49]:
# Step 1: Get all exercises that were not complete in the scrapper
non_complete_exercises = extract_non_complete_exercises_from_json()

['dumbbell-chest-fly', 'dumbbell-incline-press', 'machine-chest-fly', 'standing-cable-chest-fly', 'standing-resistance-band-chest-fly', 'barbell-upright-row', 'dumbbell-lateral-raise', 'face-pull', 'overhead-press', 'reverse-cable-fly', 'reverse-dumbbell-flyes', 'seated-dumbbell-shoulder-press', 'barbell-curl', 'barbell-preacher-curl', 'dumbbell-curl', 'hammer-curl', 'barbell-standing-triceps-extension', 'barbell-lying-triceps-extension', 'overhead-cable-triceps-extension', 'tricep-pushdown-with-bar', 'tricep-pushdown-with-rope', 'bulgarian-split-squat', 'goblet-squat', 'leg-extension', 'leg-press', 'pause-squat', 'seated-leg-curl', 'squat', 'zercher-squat', 'back-extension', 'banded-muscle-up', 'barbell-row', 'deadlift', 'dumbbell-shrug', 'jumping-muscle-up', 'lat-pulldown-with-pronated-grip', 'bar-muscle-up', 'ring-muscle-up', 'pause-deadlift', 'rack-pull', 'rowing-machine', 'stationary-bike']
42


In [55]:
# Step 2: Check which of this exercises are still to be registered
non_complete_exercises_to_be_registered = [item for item in non_complete_exercises if item not in registered_exercises]

print(non_complete_exercises_to_be_registered)
print(len(non_complete_exercises_to_be_registered))

['goblet-squat', 'squat', 'rack-pull']
3


In [56]:
# Goblet squat gives recitation error, so we'll need to erase it and create it manually later
#non_complete_exercises_to_be_registered.remove('goblet-squat')
#non_complete_exercises_to_be_registered.remove('rack-pull')

# Squat is already registered as air_squat, so we also need to remove it
non_complete_exercises_to_be_registered.remove('squat')

print(non_complete_exercises_to_be_registered)


['rack-pull']


In [57]:
process_exercise_codes(non_complete_exercises_to_be_registered)

/var/folders/32/gbbjsz7x4wq89kgmqg3whkz800vn8t/T/ipykernel_55817/3957142109.py:3: DeprecationWarning: invalid escape sequence '\/'
  html_as_string = file.read().encode('raw_unicode_escape').decode('unicode_escape').encode('utf-16_BE','surrogatepass').decode('utf-16_BE')


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "RECITATION",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.20914645,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.08601588
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.32917607,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.05975658
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.13846178,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.046378203
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "probability": "

#### 2. Work on exercises from scrapper JSON
Let's now work on the exercises for which all strengthlog info was added to the scrapper, but we want to complement.

In [12]:
registered_exercises = list_registered_exercises()

# Add in registered_exercises those that are uncomplete in the scrapper and I want to produce with the HTML next week
registered_exercises.extend(['goblet-squat', 'rack-pull'])

len(registered_exercises)

['smith-machine-squat', 'trap-bar-deadlift-with-high-handles', 'body-weight-lunge', 'one-handed-cable-row', 'inverted-row-with-underhand-grip', 'snatch-grip-deadlift', 'dumbbell-deadlift', 'ball-slams', 'chin-up', 'triceps-bodyweight-extension', 'floor-press', 'dumbbell-wrist-curl', 'machine-chest-fly', 'dumbbell-lying-triceps-extension', 'lying-windshield-wiper', 'dumbbell-decline-chest-press', 'standing-glute-kickback-in-machine', 'reverse-hyperextension', 'good-morning', 'clamshells', 'lateral-walk-with-band', 'dumbbell-pullover', 'hammer-curl', 'dumbbell-row', 'stiff-legged-deadlift', 'machine-glute-kickbacks', 'crunch', 'seated-dumbbell-shoulder-press', 'bulgarian-split-squat', 'plate-pinch', 'barbell-wrist-curl', 'one-handed-lat-pulldown', 'block-snatch', 'block-clean', 'cable-close-grip-seated-row', 'barbell-walking-lunges', 'dumbbell-horizontal-external-shoulder-rotation', 'bodyweight-curl', 'bar-muscle-up', 'stationary-bike', 'seated-smith-machine-shoulder-press', 'shallow-lun

243

In [13]:
all_exercises = list_all_exercises_from_scrapper_json()

['band-assisted-bench-press', 'bar-dip', 'bench-press', 'bench-press-against-band', 'board-press', 'cable-chest-press', 'close-grip-bench-press', 'close-grip-feet-up-bench-press', 'decline-bench-press', 'decline-push-up', 'dumbbell-chest-fly', 'dumbbell-chest-press', 'dumbbell-decline-chest-press', 'dumbbell-floor-press', 'dumbbell-pullover', 'feet-up-bench-press', 'floor-press', 'incline-bench-press', 'dumbbell-incline-press', 'incline-push-up', 'kettlebell-floor-press', 'kneeling-incline-push-up', 'kneeling-push-up', 'machine-chest-fly', 'machine-chest-press', 'pec-deck', 'pin-bench-press', 'push-up', 'push-up-against-wall', 'push-ups-with-feet-in-rings', 'resistance-band-chest-fly', 'smith-machine-bench-press', 'smith-machine-incline-bench-press', 'standing-cable-chest-fly', 'standing-resistance-band-chest-fly', 'band-external-shoulder-rotation', 'band-internal-shoulder-rotation', 'band-pull-apart', 'barbell-front-raise', 'barbell-rear-delt-row', 'barbell-upright-row', 'behind-the-n

In [14]:
remaining_exercises_list = [item for item in all_exercises if item not in registered_exercises]

# Remove squat, which is already registered as air-squat
remaining_exercises_list.remove('squat')

len(remaining_exercises_list)

2

In [68]:
def process_remaining_exercises(names):
    """
    Process all remaining exercise codes by generating an improved version of the JSON produced with the scrapper
    and saving it to a specific directory. Then, remove these processed codes from the list.
    
    Args:
    - names: A list of exercise codes.
    """
    # Do it for just 15 exercises
    for exercise_code in names[:15]:
        # Step 1: Generate new JSON from original scrapped one
        generated_json = gen_json_based_on_scrapper(exercise_code)
        
        # Step 2: Save the generated JSON to a file in the specified directory
        output_path = os.path.join('data', 'jsons', f'{exercise_code}.json')
        with open(output_path, 'w') as json_file:
            json_file.write(generated_json)
            
        print(f'JSON Generated: {exercise_code}.json')
        names.remove(exercise_code) # Delete processed exercise from remaining exercises list

    # Remove the first 15 processed codes from the list
    del names[:15]

process_remaining_exercises(remaining_exercises_list)

len(remaining_exercises_list)

JSON Generated: dumbbell-pullover.json
JSON Generated: incline-push-up.json
JSON Generated: machine-chest-press.json
JSON Generated: push-up-against-wall.json
JSON Generated: smith-machine-incline-bench-press.json
JSON Generated: barbell-front-raise.json
JSON Generated: cable-rear-delt-row.json
JSON Generated: dumbbell-rear-delt-row.json
JSON Generated: lying-dumbbell-internal-shoulder-rotation.json
JSON Generated: plate-front-raise.json
JSON Generated: seated-barbell-overhead-press.json
JSON Generated: split-jerk.json
JSON Generated: concentration-curl.json
JSON Generated: spider-curl.json
JSON Generated: dumbbell-standing-triceps-extension.json


31